### Ячейка 1: Импорты

In [337]:
# -*- coding: utf-8 -*-
# =============================================================================
# Ячейка 1: Импорты и Базовая Настройка
# =============================================================================
print("--- Ячейка 1: Импорты и Базовая Настройка ---")

# --- Базовые библиотеки ---
import numpy as np
import pandas as pd
import os
os.environ['KMP_DUPLICATE_LIB_OK']='TRUE' # Для избежания конфликтов на некоторых системах
import zipfile
import time
import warnings
import json
import random
import traceback
from pathlib import Path
from typing import Union, Tuple, List, Dict, Optional

# --- Аудио и Сигналы ---
import librosa
import librosa.display

# --- Визуализация и Интерактивность (IPython/Jupyter) ---
try:
    import ipywidgets as widgets
    from IPython.display import display, Audio, clear_output
    IPYWIDGETS_AVAILABLE = True
except ImportError:
    IPYWIDGETS_AVAILABLE = False
    print("Предупреждение: ipywidgets не найден. Интерактивные ячейки будут недоступны.")
    def display(*args, **kwargs): pass
    def Audio(*args, **kwargs): pass
    def clear_output(*args, **kwargs): pass

import matplotlib.pyplot as plt
# import scipy.signal as signal # Больше не используется

# --- PyTorch ---
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split, Subset
from torch.nn.utils.rnn import pad_sequence
import torch.nn.functional as F
from torch.optim.lr_scheduler import OneCycleLR # Импортируем сразу

# --- Метрики и Утилиты ---
import Levenshtein
from tqdm.notebook import tqdm # Используем версию для Jupyter
import itertools

# --- Логирование Экспериментов (MLflow) ---
try:
    import mlflow
    import mlflow.pytorch
    MLFLOW_AVAILABLE = True
except ImportError:
    MLFLOW_AVAILABLE = False
    print("Предупреждение: mlflow не найден. Логирование экспериментов будет отключено.")
    # Заглушка для MLflow будет определена в Ячейке 12

# --- Настройка окружения и предупреждений ---
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

print("\n--- Статус ключевых библиотек ---")
print(f"PyTorch Версия: {torch.__version__}")
print(f"LibROSA Версия: {librosa.__version__}")
print(f"NumPy Версия: {np.__version__}")
print(f"Pandas Версия: {pd.__version__}")
print(f"Levenshtein Версия: {Levenshtein.__version__}")
print(f"MLflow доступен: {MLFLOW_AVAILABLE}")
print(f"IPyWidgets доступен: {IPYWIDGETS_AVAILABLE}")
print("-" * 50)
# =============================================================================

--- Ячейка 1: Импорты и Базовая Настройка ---

--- Статус ключевых библиотек ---
PyTorch Версия: 2.5.1+cu121
LibROSA Версия: 0.11.0
NumPy Версия: 2.0.2
Pandas Версия: 2.2.3
Levenshtein Версия: 0.27.1
MLflow доступен: True
IPyWidgets доступен: True
--------------------------------------------------


### Ячейка 2: Конфигурация и Параметры

In [338]:
# -*- coding: utf-8 -*-
# =============================================================================
# Ячейка 2: Конфигурация и Параметры - ОБНОВЛЕНА (Добавлена Simple Model Config)
# =============================================================================
print("--- Ячейка 2: Конфигурация и Параметры (ОБНОВЛЕНА для Simple Model) ---")

import json
from pathlib import Path
import random
import numpy as np
import torch

SEED = 42

# --- Пути ---
BASE_DIR = Path('.').resolve()
DATA_DIR = BASE_DIR
ZIP_PATH = DATA_DIR / 'morse_dataset.zip'
AUDIO_DIR_NAME = 'morse_dataset'
EXTRACTED_AUDIO_DIR = DATA_DIR / AUDIO_DIR_NAME / AUDIO_DIR_NAME
TRAIN_CSV_PATH = DATA_DIR / 'train.csv'
TEST_CSV_PATH = DATA_DIR / 'test.csv'
SAMPLE_SUB_PATH = DATA_DIR / 'sample_submission.csv'
OUTPUT_DIR = BASE_DIR / 'output_melspec_experiments'
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

print(f"Базовая директория: {BASE_DIR}")
print(f"Ожидаемая директория аудио: {EXTRACTED_AUDIO_DIR}")
print(f"Директория для вывода: {OUTPUT_DIR}")

# --- Параметры обработки аудио (Базовые для теста n_mels) ---
SR_FOR_CONFIG = 8000
HOP_LENGTH_CONST = 64 # Фиксированный hop_length
N_MELS_DEFAULT = 16 # Дефолтное значение (будет переопределяться)

AUDIO_CONFIG_MELSPEC = {
    "feature_type": "melspec", "sample_rate": SR_FOR_CONFIG, "n_fft": 128,
    "hop_length": HOP_LENGTH_CONST, "n_mels": N_MELS_DEFAULT, # Будет [8, 12] в цикле
    "fmin": 0.0, "fmax": 4200.0, "power": 2.0,
    "apply_trimming": False, "trim_top_db": 30,
}
print(f"\nАудио параметры (Базовые для теста n_mels, hop={HOP_LENGTH_CONST}):")
print(f"  (n_mels будет [8, 12], f_range={AUDIO_CONFIG_MELSPEC['fmin']:.0f}-{AUDIO_CONFIG_MELSPEC['fmax']:.0f}Hz)")
print(json.dumps(AUDIO_CONFIG_MELSPEC, indent=2))

# --- Параметры Модели (Полная версия - для справки) ---
MODEL_CONFIG_FULL = {
    "input_feature_dim": N_MELS_DEFAULT, # Будет [8, 12] в цикле
    "cnn_out_channels": [64, 128, 128], "cnn_kernel_size": 9, "cnn_stride": 1, "cnn_padding": 'same',
    "cnn_pool_kernel": 2, "rnn_hidden_size": 512, "rnn_num_layers": 3, "dropout_rate": 0.2,
    "activation_fn": "GELU", "classifier_type": "single", "bidirectional": True # Полная модель - BiGRU
}
print(f"\nПараметры ПОЛНОЙ модели (для справки):")
print(json.dumps(MODEL_CONFIG_FULL, indent=2))

# --- Параметры Модели (УПРОЩЕННАЯ версия для калибровки) ---
MODEL_CONFIG_SIMPLE = {
    "input_feature_dim": N_MELS_DEFAULT, # Будет [8, 12] в цикле
    "cnn_out_channels": [64, 128, 128],       # Уменьшено: 2 слоя CNN, меньше каналов
    "cnn_kernel_size": 9, "cnn_stride": 1, "cnn_padding": 'same',
    "cnn_pool_kernel": 2,
    "rnn_hidden_size": 512,             # Уменьшено: размер RNN
    "rnn_num_layers": 2,                # Уменьшено: 1 слой RNN
    "dropout_rate": 0.2,
    "activation_fn": "GELU", "classifier_type": "single",
    "bidirectional": True             # <-- Упрощено: Однонаправленный GRU
}
print(f"\nПараметры УПРОЩЕННОЙ модели (для калибровки n_mels):")
print(json.dumps(MODEL_CONFIG_SIMPLE, indent=2))

# --- Параметры Обучения (Оставляем прежними, но можно уменьшить num_epochs) ---
TRAIN_CONFIG_MELSPEC = {
    "batch_size": 8, "num_workers": 0, "num_epochs": 10, # <-- Уменьшено кол-во эпох для скорости
    "learning_rate": 5e-4, "div_factor": 3.0, "final_div_factor": 100,
    "weight_decay": 1e-4, "optimizer": "AdamW", "early_stopping_patience": 5, # <-- Уменьшена терпимость
    "gradient_clip_norm": 2.0, "validation_split_ratio": 0.1,
    "base_seed": SEED, "batches_per_epoch": 1000 # Полные эпохи
}
print(f"\nПараметры обучения (Эпохи/Терпимость уменьшены):")
print(json.dumps(TRAIN_CONFIG_MELSPEC, indent=2))

# --- Специальные Токены и Глобальные Переменные ---
PAD_TOKEN = '<pad>'; BLANK_TOKEN = '_'
PAD_IDX = -1; BLANK_IDX = -1; NUM_CLASSES_CTC = -1
char_to_index: Dict[str, int] = {}; index_to_char: Dict[int, str] = {}

# --- Шаблон Базового Суффикса Имени Файла (для УПРОЩЕННОЙ модели) - ОБНОВЛЕН ---
BASE_FILENAME_SUFFIX_TEMPLATE_SIMPLE = (
    f"SR{AUDIO_CONFIG_MELSPEC['sample_rate'] // 1000}k_"
    f"MelSpec{{n_mels}}_fft{AUDIO_CONFIG_MELSPEC['n_fft']}h{HOP_LENGTH_CONST}_f0-4k2_" # {n_mels} - плейсхолдер
    f"CNN{MODEL_CONFIG_SIMPLE['cnn_out_channels'][-1]}_" # Каналы из SIMPLE
    f"RNN{MODEL_CONFIG_SIMPLE['rnn_num_layers']}x{MODEL_CONFIG_SIMPLE['rnn_hidden_size']}_GRU_" # Параметры и тип RNN из SIMPLE
    f"{MODEL_CONFIG_SIMPLE['activation_fn']}_Cls{MODEL_CONFIG_SIMPLE['classifier_type']}_"
    f"LR{TRAIN_CONFIG_MELSPEC['learning_rate']:.0e}_WD{TRAIN_CONFIG_MELSPEC['weight_decay']:.0e}"
)
print(f"\nШаблон базового суффикса (УПРОЩЕННАЯ модель): {BASE_FILENAME_SUFFIX_TEMPLATE_SIMPLE}")

# --- Выбор устройства и Установка SEED ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"\nИспользуемое устройство: {device}")
if device.type == 'cuda': print(f"  GPU: {torch.cuda.get_device_name(0)}"); torch.cuda.empty_cache()

def set_seed(seed_value: int):
    """Устанавливает seed для воспроизводимости."""
    random.seed(seed_value); np.random.seed(seed_value); torch.manual_seed(seed_value)
    if torch.cuda.is_available(): torch.cuda.manual_seed_all(seed_value)
    print(f"Установлен SEED = {seed_value}")
set_seed(TRAIN_CONFIG_MELSPEC['base_seed'])

print("\n--- Ячейка 2: Конфигурация для тестов n_mels = [8, 12] (УПРОЩЕННАЯ модель) готова ---")
print("-" * 50)
# =============================================================================

--- Ячейка 2: Конфигурация и Параметры (ОБНОВЛЕНА для Simple Model) ---
Базовая директория: C:\Users\vasja\OneDrive\Рабочий стол\Morse_mel\MorseAudioDecoder
Ожидаемая директория аудио: C:\Users\vasja\OneDrive\Рабочий стол\Morse_mel\MorseAudioDecoder\morse_dataset\morse_dataset
Директория для вывода: C:\Users\vasja\OneDrive\Рабочий стол\Morse_mel\MorseAudioDecoder\output_melspec_experiments

Аудио параметры (Базовые для теста n_mels, hop=64):
  (n_mels будет [8, 12], f_range=0-4200Hz)
{
  "feature_type": "melspec",
  "sample_rate": 8000,
  "n_fft": 128,
  "hop_length": 64,
  "n_mels": 16,
  "fmin": 0.0,
  "fmax": 4200.0,
  "power": 2.0,
  "apply_trimming": false,
  "trim_top_db": 30
}

Параметры ПОЛНОЙ модели (для справки):
{
  "input_feature_dim": 16,
  "cnn_out_channels": [
    64,
    128,
    128
  ],
  "cnn_kernel_size": 9,
  "cnn_stride": 1,
  "cnn_padding": "same",
  "cnn_pool_kernel": 2,
  "rnn_hidden_size": 512,
  "rnn_num_layers": 3,
  "dropout_rate": 0.2,
  "activation_fn": "

### Ячейка 3: Загрузка данных и распаковка

In [339]:
# -*- coding: utf-8 -*-
# =============================================================================
# Ячейка 3: Загрузка Метаданных и Распаковка Аудио
# =============================================================================
print("--- Ячейка 3: Загрузка метаданных и распаковка аудио ---")

# --- Проверка и Распаковка Архива ---
if not EXTRACTED_AUDIO_DIR.exists():
    print(f"Папка для аудио ({EXTRACTED_AUDIO_DIR}) не найдена.")
    if ZIP_PATH.is_file():
        print(f"Найден архив: {ZIP_PATH}. Распаковка...")
        try:
            with zipfile.ZipFile(ZIP_PATH, 'r') as zip_ref: zip_ref.extractall(DATA_DIR)
            print(f"Архив распакован в: {DATA_DIR}")
            if not EXTRACTED_AUDIO_DIR.is_dir(): raise FileNotFoundError(f"Не удалось найти папку '{EXTRACTED_AUDIO_DIR.name}' после распаковки в {DATA_DIR}.")
            else: print(f"Папка с аудио найдена: {EXTRACTED_AUDIO_DIR}")
        except Exception as e: print(f"Критическая ошибка при распаковке: {e}"); traceback.print_exc(); raise SystemExit("Остановка.")
    else: print(f"Критическая ошибка: Архив {ZIP_PATH} не найден."); raise SystemExit("Остановка.")
else: print(f"Папка с аудио ({EXTRACTED_AUDIO_DIR}) уже существует. Распаковка пропускается.")

# --- Загрузка CSV файлов метаданных ---
print("\nЗагрузка CSV файлов...")
try:
    train_df = pd.read_csv(TRAIN_CSV_PATH)
    print(f"  Train DataFrame загружен: {len(train_df)} записей, Колонки: {train_df.columns.tolist()}")
    test_df = pd.read_csv(TEST_CSV_PATH)
    print(f"  Test DataFrame загружен: {len(test_df)} записей, Колонки: {test_df.columns.tolist()}")
    try: sample_sub_df = pd.read_csv(SAMPLE_SUB_PATH); print(f"  Sample Submission загружен: {len(sample_sub_df)} записей.")
    except FileNotFoundError: print(f"  Предупреждение: Файл Sample Submission ({SAMPLE_SUB_PATH}) не найден."); sample_sub_df = None
except FileNotFoundError as e: print(f"Критическая ошибка: Не найден CSV файл: {e}."); raise SystemExit("Остановка.")
except Exception as e: print(f"Критическая ошибка при чтении CSV: {e}"); traceback.print_exc(); raise SystemExit("Остановка.")

# --- Проверка наличия файлов и колонок ---
print("\nПроверка данных...")
if 'id' not in train_df.columns or 'message' not in train_df.columns: raise ValueError("В train_df нет 'id' или 'message'.")
if 'id' not in test_df.columns: raise ValueError("В test_df нет 'id'.")

# --- Выборочная проверка существования аудиофайлов ---
print("\nВыборочная проверка наличия аудиофайлов...")
if EXTRACTED_AUDIO_DIR.is_dir():
    num_check = 5; example_ids = train_df['id'].sample(min(num_check, len(train_df)), random_state=SEED).tolist(); missing_files = []
    for file_id in example_ids:
        if not (EXTRACTED_AUDIO_DIR / file_id).is_file(): missing_files.append(file_id)
    if not missing_files: print(f"  Проверено {len(example_ids)} файлов - все найдены (например, {EXTRACTED_AUDIO_DIR / example_ids[0]}).")
    else: print(f"  !!! ПРЕДУПРЕЖДЕНИЕ: Не найдены файлы для ID: {missing_files} !!!")
else: print(f"  Проверка невозможна: Папка {EXTRACTED_AUDIO_DIR} не существует.")

print("\n--- Ячейка 3: Загрузка данных завершена ---")
print("-" * 50)
# =============================================================================

--- Ячейка 3: Загрузка метаданных и распаковка аудио ---
Папка с аудио (C:\Users\vasja\OneDrive\Рабочий стол\Morse_mel\MorseAudioDecoder\morse_dataset\morse_dataset) уже существует. Распаковка пропускается.

Загрузка CSV файлов...
  Train DataFrame загружен: 30000 записей, Колонки: ['id', 'message']
  Test DataFrame загружен: 5000 записей, Колонки: ['id']
  Sample Submission загружен: 5000 записей.

Проверка данных...

Выборочная проверка наличия аудиофайлов...
  Проверено 5 файлов - все найдены (например, C:\Users\vasja\OneDrive\Рабочий стол\Morse_mel\MorseAudioDecoder\morse_dataset\morse_dataset\2309.opus).

--- Ячейка 3: Загрузка данных завершена ---
--------------------------------------------------


### Ячейка 4: Создание словаря символов

In [340]:
# -*- coding: utf-8 -*-
# =============================================================================
# Ячейка 4: Создание Словаря Символов
# =============================================================================
print("--- Ячейка 4: Создание словаря символов ---")

if 'train_df' not in globals() or train_df is None: raise SystemExit("Остановка: train_df не найден.")
if 'message' not in train_df.columns: raise SystemExit("Остановка: В train_df нет 'message'.")

try:
    # Собираем все уникальные символы из обучающей выборки
    all_texts = train_df['message'].fillna('').astype(str)
    unique_chars = sorted(list(set(char for text in all_texts for char in text)))

    # Создаем словари для преобразования
    char_to_index = {char: i for i, char in enumerate(unique_chars)}
    index_to_char = {i: char for char, i in char_to_index.items()}

    # Добавляем специальные токены BLANK (для CTC) и PAD (для выравнивания)
    BLANK_IDX = len(char_to_index)
    PAD_IDX = len(char_to_index) + 1
    char_to_index[BLANK_TOKEN] = BLANK_IDX
    char_to_index[PAD_TOKEN] = PAD_IDX
    index_to_char[BLANK_IDX] = BLANK_TOKEN
    index_to_char[PAD_IDX] = PAD_TOKEN

    # Общее количество классов для выходного слоя модели (включая BLANK)
    NUM_CLASSES_CTC = BLANK_IDX + 1

    print(f"Найдено уникальных символов: {len(unique_chars)}")
    print(f"Размер словаря (включая BLANK и PAD): {len(char_to_index)}")
    print(f"Индекс BLANK ('{BLANK_TOKEN}'): {BLANK_IDX}, Индекс PAD ('{PAD_TOKEN}'): {PAD_IDX}")
    print(f"Количество классов для CTC Loss: {NUM_CLASSES_CTC}")

except Exception as e:
    print(f"Критическая ошибка при создании словаря: {e}")
    traceback.print_exc()
    raise SystemExit("Остановка.")

if not char_to_index or not index_to_char or BLANK_IDX == -1 or PAD_IDX == -1 or NUM_CLASSES_CTC <= 0:
    raise SystemExit("Остановка: Ошибка инициализации словаря.")

print("\n--- Ячейка 4: Создание словаря завершено ---")
print("-" * 50)
# =============================================================================

--- Ячейка 4: Создание словаря символов ---
Найдено уникальных символов: 44
Размер словаря (включая BLANK и PAD): 46
Индекс BLANK ('_'): 44, Индекс PAD ('<pad>'): 45
Количество классов для CTC Loss: 45

--- Ячейка 4: Создание словаря завершено ---
--------------------------------------------------


### Ячейка 5: Класс MorseDataset

In [341]:
# -*- coding: utf-8 -*-
# =============================================================================
# Ячейка 5: Класс MorseDataset (Мел-Спектрограммы)
# =============================================================================
print("--- Ячейка 5: Определение класса MorseDataset (Мел-Спектрограммы) ---")

# Проверка наличия глобальных переменных из предыдущих ячеек
if 'BLANK_IDX' not in globals() or 'PAD_IDX' not in globals(): raise ValueError("Индексы BLANK/PAD не инициализированы.")
if 'AUDIO_CONFIG_MELSPEC' not in globals(): raise ValueError("AUDIO_CONFIG_MELSPEC не определен.")
# MODEL_CONFIG_MELSPEC проверяется при создании модели

class MorseDataset(Dataset):
    """
    Датасет для Морзе (Мел-Спектрограммы).
    Выполняет: Загрузка -> Ресэмплинг -> [Trimming] -> Мел-Спектрограмма -> dB -> Z-Score.
    """
    def __init__(self,
                 dataframe: pd.DataFrame,
                 audio_dir: Path,
                 char_to_index: Dict[str, int],
                 audio_config: Dict, # Принимает актуальный audio_config для запуска
                 model_input_feature_dim: int, # Принимает актуальный n_mels
                 is_train: bool = True):
        super().__init__()
        if not isinstance(dataframe, pd.DataFrame): raise TypeError("dataframe должен быть pandas DataFrame")
        if not isinstance(audio_dir, Path): raise TypeError("audio_dir должен быть pathlib.Path")
        if not isinstance(char_to_index, dict): raise TypeError("char_to_index должен быть словарем")
        if not isinstance(audio_config, dict): raise TypeError("audio_config должен быть словарем")

        self.dataframe = dataframe.reset_index(drop=True)
        self.audio_dir = audio_dir
        self.char_to_index = char_to_index
        self.is_train = is_train
        self.audio_config = audio_config
        self.expected_feature_dim = model_input_feature_dim # Ожидаемый размер = n_mels

        # --- Извлечение параметров из audio_config ---
        try:
            self.feature_type = self.audio_config.get('feature_type', 'melspec')
            if self.feature_type != 'melspec': raise ValueError(f"Поддерживается только feature_type='melspec'")

            self.sample_rate = int(self.audio_config['sample_rate'])
            self.n_fft = int(self.audio_config['n_fft'])
            self.hop_length = int(self.audio_config['hop_length'])
            self.n_mels = int(self.audio_config['n_mels'])
            self.power = float(self.audio_config.get('power', 2.0))
            self.fmin = float(self.audio_config.get('fmin', 0.0))
            self.fmax = float(self.audio_config.get('fmax', self.sample_rate / 2.0))
            self.apply_trimming = bool(self.audio_config.get('apply_trimming', False))
            self.trim_top_db = float(self.audio_config.get('trim_top_db', 30))

            nyquist = self.sample_rate / 2.0
            if self.fmax > nyquist + 1:
                print(f"  Предупреждение Dataset: fmax ({self.fmax:.0f} Hz) > Найквиста ({nyquist:.0f} Hz).")

        except KeyError as e: raise ValueError(f"Отсутствует ключ в audio_config: {e}")
        except (TypeError, ValueError) as e: raise ValueError(f"Ошибка типа/значения в audio_config: {e}")

        # --- Валидация параметров ---
        if self.sample_rate <= 0: raise ValueError("sample_rate > 0")
        if self.n_fft <= 0: raise ValueError("n_fft > 0")
        if self.hop_length <= 0: raise ValueError("hop_length > 0")
        if self.n_mels <= 0: raise ValueError("n_mels > 0")
        if self.apply_trimming and self.trim_top_db <= 0: raise ValueError("trim_top_db > 0 при apply_trimming=True")
        if self.n_mels != self.expected_feature_dim:
             # Эта проверка важна при динамическом изменении n_mels
             raise ValueError(f"n_mels ({self.n_mels}) в audio_config не совпадает с model_input_feature_dim ({self.expected_feature_dim})!")

        # --- Информационное сообщение (однократно при инициализации) ---
        # Чтобы не засорять вывод при создании DataLoader, можно вывести это сообщение
        # один раз перед циклом обучения или убрать его совсем.
        # trim_status = f"Trimming ON (top_db={self.trim_top_db})" if self.apply_trimming else "Trimming OFF"
        # print(f"MorseDataset (MelSpec): is_train={self.is_train}, SR={self.sample_rate}Hz, {trim_status}")
        # print(f"  Признаки: MelSpec(n_fft={self.n_fft}, hop={self.hop_length}, n_mels={self.n_mels}, f_range={self.fmin:.0f}-{self.fmax:.0f}Hz) -> dB -> Z-Score")
        # print(f"  Нормализация: Z-Score (глобальная)")

    def __len__(self) -> int:
        """Возвращает количество примеров в датасете."""
        return len(self.dataframe)

    def _normalize_feature(self, feature_matrix: np.ndarray) -> np.ndarray:
        """Стандартная Z-score нормализация для всей матрицы признаков."""
        if not isinstance(feature_matrix, np.ndarray) or feature_matrix.size == 0:
            # Возвращаем пустой массив правильной формы (F, 0)
            return np.array([[]], dtype=np.float32).reshape(self.expected_feature_dim, 0)
        epsilon = 1e-8
        mean = np.mean(feature_matrix)
        std = np.std(feature_matrix)
        if std < epsilon:
            # Если стандартное отклонение близко к нулю, возвращаем нули
            return np.zeros_like(feature_matrix, dtype=np.float32)
        return ((feature_matrix - mean) / (std + epsilon)).astype(np.float32)

    def _calculate_features(self, waveform_np: np.ndarray, file_id_for_log: str = "N/A") -> Optional[torch.Tensor]:
        """Вычисляет признаки: Мел-Спектрограмма -> dB -> Z-Score."""
        # Ожидаем пустой тензор формы (n_mels, 0)
        empty_tensor = torch.empty((self.expected_feature_dim, 0), dtype=torch.float32)
        if not isinstance(waveform_np, np.ndarray) or waveform_np.size == 0:
            return empty_tensor
        try:
            processed_waveform = waveform_np.astype(np.float32)

            # 1. Вычисление Мел-спектрограммы
            mel_spectrogram = librosa.feature.melspectrogram(
                y=processed_waveform,
                sr=self.sample_rate,
                n_fft=self.n_fft,
                hop_length=self.hop_length,
                n_mels=self.n_mels,
                fmin=self.fmin,
                fmax=self.fmax,
                power=self.power
            )

            # 2. Конвертация в децибелы
            mel_spectrogram_db = librosa.power_to_db(mel_spectrogram, ref=np.max, top_db=None)

            # 3. Нормализация Z-Score (глобальная)
            normalized_mel_spec = self._normalize_feature(mel_spectrogram_db)

            # 4. Проверка размерности и содержимого
            if normalized_mel_spec.shape[0] != self.expected_feature_dim:
                 # Эта ошибка не должна возникать, если n_mels == expected_feature_dim
                 print(f"ERROR ({file_id_for_log}): Неожиданное кол-во признаков! Ожидалось {self.expected_feature_dim}, получено {normalized_mel_spec.shape[0]}")
                 return None

            features_tensor = torch.from_numpy(normalized_mel_spec) # Уже (F, T)

            if not torch.isfinite(features_tensor).all():
                # Заменяем NaN/Inf на 0, чтобы не терять пример
                print(f"WARNING ({file_id_for_log}): NaN/Inf в тензоре Мел-спектрограммы! Замена на 0.")
                features_tensor = torch.nan_to_num(features_tensor, nan=0.0, posinf=0.0, neginf=0.0)

            # Проверка на пустой тензор после всех операций
            if features_tensor.shape[1] == 0:
                # print(f"Debug ({file_id_for_log}): Пустой тензор признаков после обработки.")
                return empty_tensor

            return features_tensor # (n_mels, Time)

        except Exception as e:
            print(f"CRITICAL ERROR в _calculate_features (MelSpec) ({file_id_for_log}): {e}")
            traceback.print_exc(limit=1)
            return None

    def __getitem__(self, index: int) -> Union[Tuple[torch.Tensor, torch.Tensor], Tuple[torch.Tensor, str], Tuple[None, Optional[str]]]:
        """Загрузка, обработка (Мел-Спектрограмма) и возврат примера."""
        if not (0 <= index < len(self.dataframe)):
            # Возвращаем None и ID для тестового режима, если индекс невалиден
            return None, (f"InvalidIndex_{index}" if not self.is_train else None)
        try:
            # Получаем строку из DataFrame и ID файла
            row = self.dataframe.iloc[index]
            file_id = row['id']
            audio_path = self.audio_dir / file_id
        except Exception as e:
            print(f"Error get item data index {index}: {e}")
            return None, (f"DataAccessError_{index}" if not self.is_train else None)

        # 1. Загрузка и Ресэмплинг
        waveform_np: Optional[np.ndarray] = None
        try:
            waveform_np, _ = librosa.load(audio_path, sr=self.sample_rate, mono=True)
            if waveform_np is None or waveform_np.size == 0:
                # Файл пуст или не удалось загрузить
                return None, (file_id if not self.is_train else None)
        except FileNotFoundError:
            print(f"ERROR ({file_id}): Файл не найден {audio_path}")
            return None, (file_id if not self.is_train else None)
        except Exception as e:
            print(f"ERROR ({file_id}): Ошибка загрузки librosa: {e}")
            traceback.print_exc(limit=1)
            return None, (file_id if not self.is_train else None)

        # 2. Обрезка тишины (Trimming) - если включено
        if self.apply_trimming:
            try:
                waveform_trimmed, _ = librosa.effects.trim(waveform_np, top_db=self.trim_top_db, frame_length=512, hop_length=128)
                if waveform_trimmed is not None and waveform_trimmed.size > 0:
                    waveform_np = waveform_trimmed
                # else: Оставляем исходный, если trim удалил всё
            except Exception as e_trim:
                print(f"ERROR ({file_id}): Ошибка librosa.effects.trim: {e_trim}. Используется исходный сигнал.")

        # 3. Вычисление признаков (Мел-Спектрограмма -> dB -> Z-Score)
        features: Optional[torch.Tensor] = self._calculate_features(waveform_np, file_id)

        # Проверка на None или пустой тензор (0 временных шагов)
        if features is None or features.shape[1] == 0:
            return None, (file_id if not self.is_train else None)

        # 4. Подготовка цели (для train) или возврат ID (для test)
        if self.is_train:
            message_text = str(row.get('message', ''))
            target_indices = [self.char_to_index.get(c) for c in message_text if c in self.char_to_index]
            # Пропускаем примеры без валидных символов в таргете
            if not target_indices:
                return None, None # Возвращаем None, None чтобы collate_fn его отфильтровал
            target_tensor = torch.tensor(target_indices, dtype=torch.long)
            return features, target_tensor # (F, T), (Target_Len)
        else:
            # Для тестового режима возвращаем признаки и ID файла
            return features, file_id # (F, T), str

print("\n--- Ячейка 5: Определение MorseDataset (Мел-Спектрограммы) завершено ---")
print("-" * 50)
# =============================================================================

--- Ячейка 5: Определение класса MorseDataset (Мел-Спектрограммы) ---

--- Ячейка 5: Определение MorseDataset (Мел-Спектрограммы) завершено ---
--------------------------------------------------


### Ячейка 6: Функция collate_fn

In [342]:
# -*- coding: utf-8 -*-
# =============================================================================
# Ячейка 6: Функция collate_fn (Сборка Батчей)
# =============================================================================
print("--- Ячейка 6: Определение функции collate_fn ---")

# Проверка зависимостей
if 'PAD_IDX' not in globals(): raise ValueError("PAD_IDX не инициализирован!")
if 'torch' not in globals(): import torch
if 'pad_sequence' not in globals(): from torch.nn.utils.rnn import pad_sequence
from typing import List, Tuple, Optional, Union

def collate_fn(batch: List[Tuple[Optional[torch.Tensor], Optional[Union[torch.Tensor, str]]]]) \
    -> Optional[Tuple[torch.Tensor, Union[torch.Tensor, List[str]], torch.Tensor, Optional[torch.Tensor]]]:
    """
    Собирает батч данных из списка кортежей, возвращаемых MorseDataset.
    Фильтрует некорректные примеры (где признаки или таргет/ID равны None).
    Выполняет паддинг признаков и таргетов (для train/val).
    Возвращает батч признаков, батч таргетов (или ID), длины признаков и длины таргетов.
    """
    # 1. Фильтрация некорректных/пустых примеров
    # Пример считается валидным, если есть тензор признаков (item[0])
    # и он не пустой по временной оси (item[0].shape[1] > 0),
    # а также есть таргет или ID (item[1]).
    valid_batch = [item for item in batch if
                   item[0] is not None and item[1] is not None and item[0].shape[1] > 0]

    # Если после фильтрации батч пуст, возвращаем None
    if not valid_batch:
        return None

    # Определяем, содержит ли батч таргеты (тензоры) или ID (строки)
    is_train_or_val_batch = isinstance(valid_batch[0][1], torch.Tensor)

    # 2. Разделение признаков и таргетов/ID
    # Признаки имеют форму (F, T), пермутируем в (T, F) для pad_sequence
    features_list = [item[0].permute(1, 0) for item in valid_batch] # Список тензоров (Ti, F)
    targets_or_ids_list = [item[1] for item in valid_batch] # Список тензоров таргетов или строк ID

    # 3. Паддинг признаков
    # batch_first=True -> (B, T_max, F)
    features_padded_time_first = pad_sequence(features_list, batch_first=True, padding_value=0.0)
    # Возвращаем к формату (B, F, T_max), ожидаемому моделью (Conv1d)
    features_padded = features_padded_time_first.permute(0, 2, 1)

    # 4. Расчет длин признаков (до паддинга)
    # Длина по временной оси (исходная T)
    feature_lengths = torch.tensor([f.shape[0] for f in features_list], dtype=torch.long)

    # 5. Обработка таргетов (для train/val) или ID (для test)
    if is_train_or_val_batch:
        # Батч содержит таргеты (тензоры)
        targets_list: List[torch.Tensor] = targets_or_ids_list
        # Паддинг таргетов
        targets_padded: torch.Tensor = pad_sequence(targets_list, batch_first=True, padding_value=PAD_IDX)
        # Расчет длин таргетов (до паддинга)
        target_lengths: torch.Tensor = torch.tensor([len(t) for t in targets_list], dtype=torch.long)
        # Возвращаем данные для обучения/валидации
        return features_padded, targets_padded, feature_lengths, target_lengths
    else:
        # Батч содержит ID файлов (строки)
        file_ids: List[str] = targets_or_ids_list
        # Возвращаем данные для инференса (таргетов и их длин нет)
        return features_padded, file_ids, feature_lengths, None

print("Функция collate_fn определена.")
print("\n--- Ячейка 6: Определение collate_fn завершено ---")
print("-" * 50)
# =============================================================================

--- Ячейка 6: Определение функции collate_fn ---
Функция collate_fn определена.

--- Ячейка 6: Определение collate_fn завершено ---
--------------------------------------------------


### Ячейка 7: Модель MorseRecognizer

In [343]:
# -*- coding: utf-8 -*-
# =============================================================================
# Ячейка 7: Модель MorseRecognizer (1D CNN + GRU/BiGRU) - ОБНОВЛЕНА
# =============================================================================
print("--- Ячейка 7: Определение модели MorseRecognizer (ОБНОВЛЕНА для GRU/BiGRU) ---")

# Проверка зависимостей
if 'MODEL_CONFIG_MELSPEC' not in globals(): raise ValueError("MODEL_CONFIG_MELSPEC не определен!") # Используем для проверки
if 'NUM_CLASSES_CTC' not in globals() or NUM_CLASSES_CTC <= 0: raise ValueError("NUM_CLASSES_CTC не корректен!")
if 'torch' not in globals(): import torch
if 'nn' not in globals(): import torch.nn as nn
from typing import Union, Tuple, List, Dict, Optional

class MorseRecognizer(nn.Module):
    """
    Модель для распознавания Морзе (1D CNN + GRU/BiGRU). - ОБНОВЛЕНА
    Принимает на вход признаки (B, F, T), где F - размерность признака (n_mels).
    """
    def __init__(self, num_classes_ctc: int, input_feature_dim: int,
                 cnn_out_channels: List[int], cnn_kernel_size: int, cnn_stride: int,
                 cnn_padding: Union[int, str], cnn_pool_kernel: int,
                 rnn_hidden_size: int, rnn_num_layers: int, dropout_rate: float,
                 activation_fn: str = "GELU", classifier_type: str = "single",
                 bidirectional: bool = True): # <-- Добавлен параметр bidirectional
        super().__init__()
        self.input_feature_dim = input_feature_dim # Сохраняем для проверки в forward
        self._time_reduction_factor = 1.0 # Фактор сжатия времени из-за MaxPool
        cnn_layers = []
        in_channels = input_feature_dim # Первый слой CNN принимает F каналов

        try:
            ActivationLayer = getattr(nn, activation_fn)
        except AttributeError:
            print(f"Warning: Activation '{activation_fn}' не найдена. Используется GELU.")
            ActivationLayer = nn.GELU

        # CNN Extractor (1D свертки по времени)
        for i, out_channels in enumerate(cnn_out_channels):
            layer = nn.Sequential(
                nn.Conv1d(in_channels, out_channels, cnn_kernel_size, cnn_stride, padding=cnn_padding),
                nn.BatchNorm1d(out_channels),
                ActivationLayer(),
                nn.MaxPool1d(cnn_pool_kernel), # Пулинг по временной оси
                nn.Dropout(dropout_rate)
            )
            cnn_layers.append(layer)
            in_channels = out_channels # Выходные каналы текущего слоя = входные для следующего
            self._time_reduction_factor *= cnn_pool_kernel # Считаем общее сжатие времени
        self.cnn_extractor = nn.Sequential(*cnn_layers)
        self.cnn_output_dim = in_channels # Размерность признаков после CNN

        # RNN (GRU или BiGRU) - ОБНОВЛЕНО
        self.rnn = nn.GRU(
            input_size=self.cnn_output_dim, # Вход RNN = выход CNN
            hidden_size=rnn_hidden_size,
            num_layers=rnn_num_layers,
            batch_first=True, # Ожидает (B, T, F_cnn)
            bidirectional=bidirectional, # <-- Используем параметр
            dropout=dropout_rate if rnn_num_layers > 1 else 0.0
        )
        # Корректный расчет выходной размерности RNN - ОБНОВЛЕНО
        rnn_output_dim = rnn_hidden_size * 2 if bidirectional else rnn_hidden_size

        # Classifier (Single или Double Linear)
        self.classifier_type = classifier_type
        if self.classifier_type == "double":
            intermediate_dim = rnn_output_dim # Можно сделать настраиваемым
            self.classifier = nn.Sequential(
                nn.Linear(rnn_output_dim, intermediate_dim),
                ActivationLayer(),
                nn.Dropout(dropout_rate),
                nn.Linear(intermediate_dim, num_classes_ctc)
            )
            classifier_str = f"DoubleLinear({rnn_output_dim}->{intermediate_dim}->{num_classes_ctc})"
        elif self.classifier_type == "single":
            # Используем корректный rnn_output_dim - ОБНОВЛЕНО
            self.classifier = nn.Linear(rnn_output_dim, num_classes_ctc)
            classifier_str = f"SingleLinear({rnn_output_dim}->{num_classes_ctc})"
        else:
            raise ValueError(f"Неизвестный classifier_type: {self.classifier_type}. Допустимы 'single', 'double'.")

        # Вывод архитектуры при инициализации (можно закомментировать для чистоты логов)
        # rnn_type_str = "BiGRU" if bidirectional else "GRU" # <-- Обновлено для лога
        # print(f"Архитектура MorseRecognizer:")
        # print(f"  Input Feature Dim: {self.input_feature_dim}")
        # print(f"  CNN (1D): {len(cnn_out_channels)} layers, OutChannels={cnn_out_channels}, Kernel={cnn_kernel_size}, Pool={cnn_pool_kernel}")
        # print(f"       Output Dim={self.cnn_output_dim}, Time Reduction Factor={self._time_reduction_factor:.1f}x")
        # print(f"  RNN: {rnn_type_str}, Layers={rnn_num_layers}, Hidden Size={rnn_hidden_size}") # <-- Обновлено для лога
        # print(f"       Output Dim={rnn_output_dim}")
        # print(f"  Activation: {activation_fn}")
        # print(f"  Classifier: {classifier_str}")
        # print(f"  Dropout: {dropout_rate}")

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Input: (B, F, T_in) - где F = input_feature_dim (n_mels)
        -> CNN -> (B, C_cnn, T_red)
        -> Permute -> (B, T_red, C_cnn)
        -> RNN -> (B, T_red, H*2 или H)
        -> Classifier -> (B, T_red, N_classes)
        -> Permute -> Output: (T_red, B, N_classes) - для CTC Loss
        """
        # Проверка размерности входа
        if x.shape[1] != self.input_feature_dim:
            raise ValueError(f"Input feature dim mismatch! Expected {self.input_feature_dim}, got {x.shape[1]}. Shape: {x.shape}")

        x = self.cnn_extractor(x)        # (B, C_cnn, T_reduced)
        x = x.permute(0, 2, 1)           # (B, T_reduced, C_cnn) - готовим для RNN (batch_first=True)
        x_rnn, _ = self.rnn(x)           # (B, T_reduced, Hidden*2 или Hidden)
        logits = self.classifier(x_rnn)  # (B, T_reduced, NumClasses)
        # Готовим для CTC Loss: (Time, Batch, Classes)
        logits = logits.permute(1, 0, 2) # (T_reduced, B, NumClasses)
        return logits

    def get_time_reduction_factor(self) -> float:
        """Возвращает фактор уменьшения временной размерности после CNN."""
        return self._time_reduction_factor

# --- Создание и Проверка Экземпляра Модели (с базовым конфигом) ---
# Эта проверка выполняется один раз для базовой конфигурации
model_created_successfully = False
model_check = None; dummy_input_check = None; dummy_output_check = None
try:
    print("\nСоздание экземпляра модели для проверки (с базовым конфигом)...")
    # Используем базовый конфиг для проверки размерностей
    # Теперь передаем bidirectional=True (как в базовом конфиге)
    model_check = MorseRecognizer(
        num_classes_ctc=NUM_CLASSES_CTC,
        bidirectional=MODEL_CONFIG_MELSPEC.get('bidirectional', True), # Используем значение из конфига или True по умолчанию
        **MODEL_CONFIG_MELSPEC # Используем базовый MODEL_CONFIG_MELSPEC
    ).to(device)
    total_params = sum(p.numel() for p in model_check.parameters() if p.requires_grad)
    print(f"Модель '{type(model_check).__name__}' создана ({total_params:,} параметров) на {device}.")

    print("\nПроверка forward pass..."); model_check.eval()
    dummy_batch_size = 2; dummy_time_steps = 500 # Уменьшим для экономии памяти
    dummy_input_check = torch.randn(dummy_batch_size, MODEL_CONFIG_MELSPEC['input_feature_dim'], dummy_time_steps).to(device)
    with torch.no_grad(): dummy_output_check = model_check(dummy_input_check)
    print(f"  Вход: {dummy_input_check.shape}, Выход: {dummy_output_check.shape}")
    expected_time_dim = int(dummy_time_steps / model_check.get_time_reduction_factor())
    if abs(dummy_output_check.shape[0] - expected_time_dim) > 2: print(f"  ПРЕДУПРЕЖДЕНИЕ: Неожиданная длина выхода! Ожидалось ~{expected_time_dim}, получено {dummy_output_check.shape[0]}.")
    assert dummy_output_check.shape[1] == dummy_batch_size, "Batch size mismatch!"
    assert dummy_output_check.shape[2] == NUM_CLASSES_CTC, "Num classes mismatch!"
    print("  Размерности выхода (T_red, B, C) корректны."); model_created_successfully = True
except Exception as e: print(f"\n!!! КРИТИЧЕСКАЯ ОШИБКА при создании/проверке модели: {e} !!!"); traceback.print_exc()
finally:
    # Очистка памяти после проверки
    if model_check is not None: del model_check
    if dummy_input_check is not None: del dummy_input_check
    if dummy_output_check is not None: del dummy_output_check
    if torch.cuda.is_available(): torch.cuda.empty_cache()

if not model_created_successfully: raise SystemExit("Остановка: Не удалось создать/проверить модель.")

print("\n--- Ячейка 7: Определение и проверка модели (ОБНОВЛЕНА для GRU/BiGRU) завершены ---")
print("-" * 50)
# =============================================================================

--- Ячейка 7: Определение модели MorseRecognizer (ОБНОВЛЕНА для GRU/BiGRU) ---

Создание экземпляра модели для проверки (с базовым конфигом)...
Модель 'MorseRecognizer' создана (11,699,181 параметров) на cuda.

Проверка forward pass...
  Вход: torch.Size([2, 16, 500]), Выход: torch.Size([62, 2, 45])
  Размерности выхода (T_red, B, C) корректны.

--- Ячейка 7: Определение и проверка модели (ОБНОВЛЕНА для GRU/BiGRU) завершены ---
--------------------------------------------------


### Ячейка 8: Loss, Optimizer, Scheduler

In [344]:
# -*- coding: utf-8 -*-
# =============================================================================
# Ячейка 8: Настройка Loss, Optimizer, Scheduler
# =============================================================================
print("--- Ячейка 8: Настройка Loss, Optimizer, Scheduler ---")

# Проверка зависимостей
if 'BLANK_IDX' not in globals(): raise ValueError("BLANK_IDX не инициализирован!")
if 'TRAIN_CONFIG_MELSPEC' not in globals(): raise ValueError("TRAIN_CONFIG_MELSPEC не определен!")
if 'optim' not in globals(): import torch.optim as optim
if 'nn' not in globals(): import torch.nn as nn
if 'OneCycleLR' not in globals(): from torch.optim.lr_scheduler import OneCycleLR

# --- Функция Потерь ---
# Используем CTCLoss, стандарт для задач sequence-to-sequence без выравнивания
criterion = nn.CTCLoss(blank=BLANK_IDX, reduction='mean', zero_infinity=True)
print(f"Функция потерь: CTCLoss (blank={BLANK_IDX}, reduction='mean', zero_infinity=True)")

# --- Оптимизатор и Планировщик (параметры из TRAIN_CONFIG_MELSPEC) ---
# Сами объекты optimizer и scheduler будут создаваться внутри run_training_pipeline
optimizer_name = TRAIN_CONFIG_MELSPEC.get('optimizer', 'AdamW').lower()
lr = TRAIN_CONFIG_MELSPEC['learning_rate']
wd = TRAIN_CONFIG_MELSPEC['weight_decay']
print(f"\nОптимизатор: {optimizer_name.upper()} (LR={lr:.1e}, WD={wd:.1e}) - будет создан в пайплайне")

div_f = TRAIN_CONFIG_MELSPEC.get('div_factor', 25.0)
final_div_f = TRAIN_CONFIG_MELSPEC.get('final_div_factor', 1e4)
print(f"Планировщик: OneCycleLR (div={div_f}, final_div={final_div_f}) - будет создан в пайплайне")

print("\n--- Ячейка 8: Настройка Loss, Optimizer, Scheduler завершена ---")
print("-" * 50)
# =============================================================================

--- Ячейка 8: Настройка Loss, Optimizer, Scheduler ---
Функция потерь: CTCLoss (blank=44, reduction='mean', zero_infinity=True)

Оптимизатор: ADAMW (LR=5.0e-04, WD=1.0e-04) - будет создан в пайплайне
Планировщик: OneCycleLR (div=3.0, final_div=100) - будет создан в пайплайне

--- Ячейка 8: Настройка Loss, Optimizer, Scheduler завершена ---
--------------------------------------------------


### Ячейка 9: Функции Декодирования и Метрики

In [345]:
# -*- coding: utf-8 -*-
# =============================================================================
# Ячейка 9: Функции Декодирования (Greedy) и Метрики (Levenshtein)
# =============================================================================
print("--- Ячейка 9: Определение функций декодирования и метрики ---")

# Проверка зависимостей
if 'index_to_char' not in globals(): raise ValueError("index_to_char не определен!")
if 'BLANK_IDX' not in globals(): raise ValueError("BLANK_IDX не определен!")
if 'PAD_IDX' not in globals(): raise ValueError("PAD_IDX не определен!")
if 'torch' not in globals(): import torch
if 'Levenshtein' not in globals(): import Levenshtein
from typing import List, Dict, Tuple

# --- Greedy CTC Decoding ---
def ctc_greedy_decode(logits: torch.Tensor, index_to_char_map: Dict[int, str], blank_idx: int) -> List[str]:
    """
    Жадное CTC декодирование батча логитов.
    Input: logits (Time, Batch, Classes) - выход модели после log_softmax не нужен.
    Output: List[str] - список декодированных строк для батча.
    """
    decoded_batch = []
    # Находим наиболее вероятный индекс для каждого временного шага в каждом примере батча
    best_path = torch.argmax(logits, dim=2) # (Time, Batch)
    best_path_np = best_path.cpu().numpy()

    # Итерируем по каждому примеру в батче
    for i in range(best_path_np.shape[1]):
        sequence_indices = best_path_np[:, i]
        # 1. Схлопываем повторяющиеся символы
        collapsed_indices = [idx for j, idx in enumerate(sequence_indices) if j == 0 or idx != sequence_indices[j-1]]
        # 2. Удаляем BLANK символы
        final_indices = [idx for idx in collapsed_indices if idx != blank_idx]
        # 3. Преобразуем индексы в символы
        decoded_string = "".join([index_to_char_map.get(idx, '?') for idx in final_indices]) # '?' для неизвестных индексов
        decoded_batch.append(decoded_string)
    return decoded_batch

# --- Levenshtein Distance ---
def calculate_levenshtein(predictions: List[str], targets_padded: torch.Tensor, target_lengths: torch.Tensor,
                          index_to_char_map: Dict[int, str], pad_idx: int) -> Tuple[float, List[Tuple[str, str]]]:
    """
    Вычисляет среднее расстояние Левенштейна для батча и возвращает пары (предсказание, реальность).
    Input:
        predictions: List[str] - список предсказанных строк.
        targets_padded: torch.Tensor (Batch, MaxTargetLen) - тензор реальных таргетов с паддингом.
        target_lengths: torch.Tensor (Batch) - тензор реальных длин таргетов.
        index_to_char_map: Dict[int, str] - словарь для преобразования индексов в символы.
        pad_idx: int - индекс PAD токена.
    Output:
        Tuple[float, List[Tuple[str, str]]] - среднее расстояние Левенштейна, список пар (предсказание, реальность).
    """
    total_distance = 0.0
    num_valid_pairs = 0
    decoded_pairs = [] # Список для хранения пар (предсказание, реальность)
    targets_np = targets_padded.cpu().numpy()
    target_lengths_np = target_lengths.cpu().numpy()
    batch_size = targets_padded.shape[0]

    # Проверка соответствия размеров предсказаний и таргетов
    if len(predictions) != batch_size:
        print(f"Warning: Levenshtein size mismatch! Preds:{len(predictions)}, Targets:{batch_size}")
        return float('inf'), [] # Возвращаем бесконечность и пустой список

    # Итерируем по каждому примеру в батче
    for i in range(batch_size):
        real_target_len = target_lengths_np[i]
        pred_str = predictions[i]

        # Получаем реальную строку таргета, удаляя паддинг
        if real_target_len <= 0:
            # Если таргет пустой (маловероятно, но возможно)
            target_str = ""
            dist = len(pred_str) # Расстояние равно длине предсказания
        else:
            target_indices = targets_np[i, :real_target_len]
            target_str = "".join([index_to_char_map.get(idx, '?') for idx in target_indices if idx != pad_idx])
            try:
                # Вычисляем расстояние Левенштейна
                dist = Levenshtein.distance(pred_str, target_str)
            except Exception as e:
                # Обработка возможных ошибок в Levenshtein
                print(f"Levenshtein Error: ('{pred_str}', '{target_str}'). {e}")
                dist = max(len(pred_str), len(target_str)) # Используем максимальную длину как штраф

        total_distance += dist
        num_valid_pairs += 1
        decoded_pairs.append((pred_str, target_str)) # Добавляем пару

    # Вычисляем среднее расстояние
    mean_levenshtein = total_distance / num_valid_pairs if num_valid_pairs > 0 else float('inf')
    return mean_levenshtein, decoded_pairs

print("Функции ctc_greedy_decode и calculate_levenshtein определены.")
print("\n--- Ячейка 9: Определение функций декодирования и метрики завершено ---")
print("-" * 50)
# =============================================================================

--- Ячейка 9: Определение функций декодирования и метрики ---
Функции ctc_greedy_decode и calculate_levenshtein определены.

--- Ячейка 9: Определение функций декодирования и метрики завершено ---
--------------------------------------------------


### Ячейка 10: Функции Обучения и Валидации Эпохи

In [346]:
# -*- coding: utf-8 -*-
# =============================================================================
# Ячейка 10: Функции Обучения и Валидации Эпохи
# =============================================================================
print("--- Ячейка 10: Определение функций обучения и валидации эпохи ---")

# Проверка зависимостей (глобальные переменные и импорты)
if 'F' not in globals(): import torch.nn.functional as F
if 'tqdm' not in globals(): from tqdm.notebook import tqdm
if 'np' not in globals(): import numpy as np
if 'torch' not in globals(): import torch
if 'optim' not in globals(): import torch.optim as optim
if 'nn' not in globals(): import torch.nn as nn
if 'traceback' not in globals(): import traceback
if 'DataLoader' not in globals(): from torch.utils.data import DataLoader
if 'itertools' not in globals(): import itertools
# Функции из Ячейки 9 должны быть определены
if 'ctc_greedy_decode' not in globals() or 'calculate_levenshtein' not in globals():
    raise NameError("Функции ctc_greedy_decode или calculate_levenshtein не определены!")
from typing import Optional, Tuple, List, Dict

# --- Функция Обучения Одной Эпохи ---
def train_epoch(model: nn.Module, dataloader: DataLoader, criterion: nn.CTCLoss, optimizer: optim.Optimizer,
                scheduler: Optional[optim.lr_scheduler._LRScheduler], device: torch.device, epoch_num: int, total_epochs: int,
                index_to_char_map: Dict[int, str], blank_idx: int, pad_idx: int, grad_clip_norm: float,
                batches_per_epoch: int = 0) -> Tuple[float, float, float]:
    """ Выполняет одну эпоху обучения модели. """
    model.train() # Переключаем модель в режим обучения
    running_loss = 0.0
    total_lev_dist = 0.0
    total_lr = 0.0
    num_batches_processed = 0
    total_samples = 0

    # Получаем фактор сжатия времени из модели (для расчета длин для CTC)
    try:
        time_factor = max(model.get_time_reduction_factor(), 1.0)
    except AttributeError:
        print("Warning: Метод get_time_reduction_factor() не найден в модели. Используется time_factor=1.0")
        time_factor = 1.0

    # Определяем итератор и количество батчей для обработки
    total_batches_in_loader = len(dataloader)
    iterator = dataloader
    num_batches_to_process = total_batches_in_loader
    if batches_per_epoch > 0 and batches_per_epoch < total_batches_in_loader:
         # Используем только часть эпохи, если batches_per_epoch задан
         iterator = itertools.islice(dataloader, batches_per_epoch)
         num_batches_to_process = batches_per_epoch
    if num_batches_to_process == 0: # Если loader пуст или batches_per_epoch=0
        print("Warning: Нет батчей для обработки в train_epoch.")
        return 0.0, float('inf'), 0.0

    # Создаем прогресс-бар
    pbar = tqdm(iterator, total=num_batches_to_process, desc=f"Эпоха {epoch_num}/{total_epochs} [Тренировка]", leave=False, ncols=1000)

    # Итерация по батчам
    for batch_idx, batch_data in enumerate(pbar):
        # Пропускаем батч, если collate_fn вернул None
        if batch_data is None: continue
        features, targets, feature_lengths, target_lengths = batch_data
        # Пропускаем, если данные некорректны
        if features is None or targets is None or feature_lengths is None or target_lengths is None: continue

        batch_size = features.size(0)
        if batch_size == 0: continue

        # Перемещаем данные на устройство
        features = features.to(device, non_blocking=True)
        targets = targets.to(device, non_blocking=True)
        # Длины оставляем на CPU для CTC Loss
        feature_lengths_cpu = feature_lengths.cpu()
        target_lengths_cpu = target_lengths.cpu()

        # Рассчитываем длины выхода модели для CTC Loss
        # Используем floor и clamp(min=1), т.к. длина не может быть 0
        input_lengths_ctc = torch.floor(feature_lengths_cpu.float() / time_factor + 1e-9).long().clamp(min=1)

        loss_value = float('inf')
        lev_dist_batch = float('inf')
        current_lr = optimizer.param_groups[0]['lr']

        try:
            # Обнуляем градиенты
            optimizer.zero_grad()

            # Прямой проход через модель
            logits = model(features) # Ожидаемый выход: (T_red, B, C)

            # Проверка размерности батча на выходе
            if logits.shape[1] != batch_size:
                print(f"\n!!! ERROR (Train): Batch size mismatch! Out:{logits.shape[1]} != In:{batch_size}. Skip batch.")
                continue

            # Применяем log_softmax для CTC Loss
            log_probs = F.log_softmax(logits, dim=2) # (T_red, B, C)

            # Убеждаемся, что длины входа для CTC не превышают реальную длину выхода модели
            output_length = log_probs.shape[0]
            input_lengths_ctc_clamped = input_lengths_ctc.clamp(max=output_length)

            # Убеждаемся, что длины таргетов не превышают размерность тензора таргетов
            target_lengths_clamped = target_lengths_cpu.clamp(max=targets.shape[1])

            # Вычисляем CTC Loss
            # Обрабатываем случай, когда target_length может быть 0 (хотя collate_fn должен это фильтровать)
            valid_target_mask = target_lengths_clamped > 0
            if not torch.all(valid_target_mask):
                 # Если есть таргеты с нулевой длиной, считаем loss только для валидных
                 log_probs_valid = log_probs[:, valid_target_mask, :]
                 targets_valid = targets[valid_target_mask, :]
                 input_lengths_valid = input_lengths_ctc_clamped[valid_target_mask]
                 target_lengths_valid = target_lengths_clamped[valid_target_mask]
                 # Считаем loss только если есть валидные примеры
                 loss = criterion(log_probs_valid, targets_valid, input_lengths_valid, target_lengths_valid) if log_probs_valid.shape[1] > 0 else torch.tensor(0.0, device=device)
            else:
                 # Все таргеты валидны
                 loss = criterion(log_probs, targets, input_lengths_ctc_clamped, target_lengths_clamped)

            # Проверка на NaN/Inf в loss
            if not torch.isfinite(loss):
                print(f"\n!!! WARNING (Train): NaN/Inf loss на батче {batch_idx}! Пропуск шага оптимизатора.")
                optimizer.zero_grad() # Сбрасываем градиенты на всякий случай
                loss_value = 30.0 # Штраф для логгирования
                lev_dist_batch = 30.0
                continue # Переходим к следующему батчу

            loss_value = loss.item()

            # Обратный проход и шаг оптимизатора
            if loss.requires_grad: # Убеждаемся, что loss требует градиентов
                loss.backward()
                # Обрезка градиентов для стабильности
                if grad_clip_norm > 0:
                    torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=grad_clip_norm)
                optimizer.step()
                # Шаг планировщика (если используется)
                if scheduler:
                    scheduler.step() # OneCycleLR обновляется после каждого батча

            # Вычисление метрики Levenshtein (без градиентов)
            with torch.no_grad():
                decoded_preds = ctc_greedy_decode(logits, index_to_char_map, blank_idx)
                lev_dist_batch, _ = calculate_levenshtein(decoded_preds, targets.cpu(), target_lengths_cpu, index_to_char_map, pad_idx)
                if not np.isfinite(lev_dist_batch): lev_dist_batch = 20.0 # Ограничиваем для логгирования

        except RuntimeError as e:
             if "CUDA out of memory" in str(e):
                 print("\n!!! CUDA Out of Memory (Train) !!! Попробуйте уменьшить batch_size.")
                 # Можно добавить логику для уменьшения batch_size или остановки
                 raise e # Перевыбрасываем ошибку
             else:
                 print(f"\n!!! RuntimeError (Train) на батче {batch_idx}: {e} !!!")
                 traceback.print_exc(limit=1)
                 loss_value = 30.0; lev_dist_batch = 30.0 # Штрафы
        except Exception as e:
             print(f"\n!!! Error (Train) на батче {batch_idx}: {e} !!!")
             traceback.print_exc(limit=1)
             loss_value = 30.0; lev_dist_batch = 30.0 # Штрафы

        # Обновление статистики эпохи
        running_loss += loss_value * batch_size
        total_lev_dist += lev_dist_batch * batch_size
        total_samples += batch_size
        total_lr += current_lr
        num_batches_processed += 1

        # Обновление прогресс-бара
        pbar.set_postfix(loss=f'{loss_value:.3f}', lev=f'{lev_dist_batch:.3f}', lr=f'{current_lr:.2e}')

    pbar.close() # Закрываем прогресс-бар

    # Вычисляем средние значения за эпоху
    avg_loss = running_loss / total_samples if total_samples > 0 else float('inf')
    avg_lev = total_lev_dist / total_samples if total_samples > 0 else float('inf')
    avg_lr = total_lr / num_batches_processed if num_batches_processed > 0 else 0.0

    return avg_loss, avg_lev, avg_lr

# --- Функция Валидации Одной Эпохи ---
def validate_epoch(model: nn.Module, dataloader: DataLoader, criterion: nn.CTCLoss, device: torch.device,
                   index_to_char_map: Dict[int, str], blank_idx: int, pad_idx: int
                  ) -> Tuple[float, float, List[Tuple[str, str]]]:
    """ Выполняет одну эпоху валидации модели. """
    model.eval() # Переключаем модель в режим оценки
    running_loss = 0.0
    total_lev_dist = 0.0
    total_samples = 0
    all_decoded_pairs = [] # Список для примеров декодирования

    # Получаем фактор сжатия времени
    try: time_factor = max(model.get_time_reduction_factor(), 1.0)
    except AttributeError: time_factor = 1.0

    # Создаем прогресс-бар
    pbar = tqdm(dataloader, desc="   [Валидация]", leave=False, ncols=1000)

    # Отключаем расчет градиентов
    with torch.no_grad():
        # Итерация по батчам
        for batch_data in pbar:
            if batch_data is None: continue
            features, targets, feature_lengths, target_lengths = batch_data
            if features is None or targets is None or feature_lengths is None or target_lengths is None: continue

            batch_size = features.size(0)
            if batch_size == 0: continue

            # Перемещаем данные на устройство
            features = features.to(device, non_blocking=True)
            targets = targets.to(device, non_blocking=True)
            feature_lengths_cpu = feature_lengths.cpu()
            target_lengths_cpu = target_lengths.cpu()

            # Рассчитываем длины для CTC
            input_lengths_ctc = torch.floor(feature_lengths_cpu.float() / time_factor + 1e-9).long().clamp(min=1)

            loss_value = float('inf')
            lev_dist_batch = float('inf')
            decoded_pairs_batch = []

            try:
                # Прямой проход
                logits = model(features) # (T_red, B, C)
                output_length = logits.shape[0]

                # Проверка размерности батча
                if logits.shape[1] != batch_size:
                    print(f"\n!!! ERROR (Val): Batch size mismatch! Out:{logits.shape[1]} != In:{batch_size}. Skip batch.")
                    continue

                # Log_softmax для CTC
                log_probs = F.log_softmax(logits, dim=2)

                # Клампинг длин
                input_lengths_ctc_clamped = input_lengths_ctc.clamp(max=output_length)
                target_lengths_clamped = target_lengths_cpu.clamp(max=targets.shape[1])

                # Расчет Loss
                valid_target_mask = target_lengths_clamped > 0
                if not torch.all(valid_target_mask):
                    log_probs_valid = log_probs[:, valid_target_mask, :]
                    targets_valid = targets[valid_target_mask, :]
                    input_lengths_valid = input_lengths_ctc_clamped[valid_target_mask]
                    target_lengths_valid = target_lengths_clamped[valid_target_mask]
                    loss = criterion(log_probs_valid, targets_valid, input_lengths_valid, target_lengths_valid) if log_probs_valid.shape[1] > 0 else torch.tensor(0.0, device=device)
                else:
                    loss = criterion(log_probs, targets, input_lengths_ctc_clamped, target_lengths_clamped)

                if torch.isfinite(loss): loss_value = loss.item()
                else: print("\nWarning (Val): NaN/Inf loss.")

                # Декодирование и расчет Levenshtein
                decoded_preds = ctc_greedy_decode(logits, index_to_char_map, blank_idx)
                lev_dist_batch, decoded_pairs_batch = calculate_levenshtein(decoded_preds, targets.cpu(), target_lengths_cpu, index_to_char_map, pad_idx)
                if not np.isfinite(lev_dist_batch): lev_dist_batch = 20.0

            except Exception as e:
                print(f"\n!!! Error (Val): {e} !!!")
                traceback.print_exc(limit=1)
                loss_value=float('inf'); lev_dist_batch=float('inf')
                decoded_pairs_batch = [("ERROR","ERROR")] * batch_size # Заполняем ошибками

            # Обновление статистики эпохи
            if np.isfinite(loss_value): # Учитываем только конечные значения loss
                running_loss += loss_value * batch_size
            total_lev_dist += lev_dist_batch * batch_size
            total_samples += batch_size

            # Сохраняем несколько примеров декодирования для вывода
            if len(all_decoded_pairs) < 10: # Сохраняем первые 10 пар
                all_decoded_pairs.extend(decoded_pairs_batch[:max(0, 10 - len(all_decoded_pairs))])

            # Обновление прогресс-бара
            pbar.set_postfix(loss=f'{loss_value:.3f}', lev=f'{lev_dist_batch:.3f}')

    pbar.close() # Закрываем прогресс-бар

    # Вычисляем средние значения за эпоху
    avg_loss = running_loss / total_samples if total_samples > 0 else float('inf')
    avg_lev = total_lev_dist / total_samples if total_samples > 0 else float('inf')

    return avg_loss, avg_lev, all_decoded_pairs

print("Функции train_epoch и validate_epoch определены.")
print("\n--- Ячейка 10: Определение функций обучения и валидации завершено ---")
print("-" * 50)
# =============================================================================

--- Ячейка 10: Определение функций обучения и валидации эпохи ---
Функции train_epoch и validate_epoch определены.

--- Ячейка 10: Определение функций обучения и валидации завершено ---
--------------------------------------------------


### Ячейка 11 Интерактивная настройка RMS

In [347]:
# -*- coding: utf-8 -*-
# =============================================================================
# Ячейка 11: Интерактивная Настройка Мел-Спектрограмм (Маленькие n_fft)
# =============================================================================
# Используем AUDIO_CONFIG_MELSPEC для параметров по умолчанию
print(f"--- Ячейка 11: Интерактивная Настройка Мел-Спектрограмм (SR={AUDIO_CONFIG_MELSPEC['sample_rate']} Гц, Маленькие n_fft) ---")
print(">>> Исследуем n_fft = [32, 64, 128, 256] <<<")

if not IPYWIDGETS_AVAILABLE:
    print("ipywidgets не доступен. Пропуск интерактивной ячейки.")
else:
    # Импорты (остаются те же)
    import ipywidgets as widgets
    from IPython.display import display, Audio, clear_output
    import matplotlib.pyplot as plt
    import librosa, librosa.display, numpy as np, pandas as pd
    from pathlib import Path; import traceback, random

    # Проверка глобальных переменных (остается та же)
    required_globals_vis = ['AUDIO_CONFIG_MELSPEC', 'EXTRACTED_AUDIO_DIR', 'train_df', 'SEED']
    missing_globals_vis = [v for v in required_globals_vis if v not in globals()]
    if missing_globals_vis: raise NameError(f"Отсутствуют переменные: {missing_globals_vis}")
    if train_df.empty: raise ValueError("train_df пуст.")

    # Базовые параметры из конфига
    SR_VIS = AUDIO_CONFIG_MELSPEC['sample_rate']
    # === ИЗМЕНЕНИЕ ЗДЕСЬ: Устанавливаем N_FFT_DEFAULT = 256 ===
    N_FFT_DEFAULT = 256
    # ========================================================
    # === ИЗМЕНЕНИЕ ЗДЕСЬ: Устанавливаем HOP_LENGTH_DEFAULT = 64 ===
    HOP_LENGTH_DEFAULT = 64 # Константа из Ячейки 2 для тренировки, используем как дефолт здесь
    # ==========================================================
    N_MELS_DEFAULT = AUDIO_CONFIG_MELSPEC['n_mels'] # n_mels пока не меняем
    FMIN_DEFAULT = AUDIO_CONFIG_MELSPEC.get('fmin', 0.0)
    FMAX_DEFAULT = AUDIO_CONFIG_MELSPEC.get('fmax', SR_VIS / 2.0) # Используем fmax=4200 из конфига
    POWER_DEFAULT = AUDIO_CONFIG_MELSPEC.get('power', 2.0)
    NYQUIST_FREQ = SR_VIS / 2.0

    print(f"Параметры визуализации (Мел-Спектрограммы): SR={SR_VIS}Hz (Найквист = {NYQUIST_FREQ:.0f} Гц)")
    print(f"  Defaults: n_fft={N_FFT_DEFAULT}, hop={HOP_LENGTH_DEFAULT}, n_mels={N_MELS_DEFAULT}, fmin={FMIN_DEFAULT:.0f}, fmax={FMAX_DEFAULT:.0f}")

    # --- Функции расчета и отрисовки (остаются без изменений) ---
    def calculate_interactive_melspec(waveform_np: np.ndarray, sr: int, n_fft: int, hop_length: int, n_mels: int, fmin: float, fmax: float, power: float, file_id_vis: str = "N/A") -> Optional[np.ndarray]:
        if not isinstance(waveform_np, np.ndarray) or waveform_np.size == 0: return None
        try: mel_spectrogram = librosa.feature.melspectrogram( y=waveform_np.astype(np.float32), sr=sr, n_fft=n_fft, hop_length=hop_length, n_mels=n_mels, fmin=fmin, fmax=fmax, power=power ); mel_spectrogram_db = librosa.power_to_db(mel_spectrogram, ref=np.max); return mel_spectrogram_db
        except Exception as e: print(f"Error calc interactive melspec ({file_id_vis}): {e}"); traceback.print_exc(limit=1); return None
    def plot_interactive_melspec(waveform: np.ndarray, mel_spec_db: np.ndarray, sr: int, hop_length: int, n_mels: int, fmin: float, fmax: float, n_fft: int, file_id: str, nyquist: float):
        fig, axes = plt.subplots(2, 1, figsize=(16, 8), sharex=True); win_ms = 1000 * n_fft / sr; hop_ms = 1000 * hop_length / sr
        title = (f"Интерактивный Анализ Мел-Спектрограммы: {file_id}\nSR={sr}, n_fft={n_fft} (~{win_ms:.1f}ms), hop={hop_length} (~{hop_ms:.1f}ms), n_mels={n_mels}, fmin={fmin:.0f}, fmax={fmax:.0f}")
        fig.suptitle(title, fontsize=14); librosa.display.waveshow(waveform, sr=sr, ax=axes[0], color='grey', alpha=0.7); axes[0].set_title("Waveform"); axes[0].label_outer(); axes[0].grid(True, linestyle=':')
        if mel_spec_db is not None and mel_spec_db.size > 0: img = librosa.display.specshow(mel_spec_db, sr=sr, hop_length=hop_length, x_axis='time', y_axis='mel', ax=axes[1], fmin=fmin, fmax=fmax, cmap='magma'); axes[1].set_title(f"Mel Spectrogram ({n_mels} bins)"); axes[1].axhline(nyquist, color='cyan', linestyle='--', linewidth=1, label=f'Найквист ({nyquist:.0f} Hz)'); axes[1].legend(loc='upper right'); fig.colorbar(img, ax=axes[1], format='%+2.0f dB')
        else: axes[1].set_title("Mel Spectrogram - Нет данных"); axes[1].text(0.5, 0.5, 'Нет данных', ha='center', va='center', transform=axes[1].transAxes)
        axes[1].set_xlabel("Время (с)"); plt.tight_layout(rect=[0, 0.03, 1, 0.93]); plt.show()

    # --- Виджеты ---
    file_ids_vis_list = train_df['id'].unique().tolist(); file_ids_vis_list = random.sample(file_ids_vis_list, min(500, len(file_ids_vis_list)))
    if not file_ids_vis_list: raise SystemExit("Остановка: Нет файлов для визуализации.")
    file_dd = widgets.Dropdown(options=file_ids_vis_list, description='Аудиофайл:', style={'description_width': 'initial'})

    # === ИЗМЕНЕНИЕ ЗДЕСЬ: Обновляем опции и значение по умолчанию для n_fft_dd ===
    n_fft_dd = widgets.Dropdown(
        options=[32, 64, 128, 256], # Новые опции
        value=N_FFT_DEFAULT, # Дефолт теперь 256
        description='n_fft:', style={'description_width': 'initial'}
    )
    # ========================================================================

    # === ИЗМЕНЕНИЕ ЗДЕСЬ: Обновляем дефолт и min/step для hop_length_slider ===
    hop_length_slider = widgets.IntSlider(
        value=HOP_LENGTH_DEFAULT, # Дефолт 64
        min=16, # Минимальный шаг
        max=N_FFT_DEFAULT, # Изначально max = дефолтный n_fft
        step=16, # Шаг поменьше
        description='hop_length:', style={'description_width': 'initial'},
        layout=widgets.Layout(width='400px'), continuous_update=False
    )
    # ======================================================================

    n_mels_slider = widgets.IntSlider(value=N_MELS_DEFAULT, min=20, max=128, step=4, description='n_mels:', style={'description_width': 'initial'}, layout=widgets.Layout(width='400px'), continuous_update=False)
    fmin_slider = widgets.IntSlider( value=int(FMIN_DEFAULT), min=0, max=1000, step=50, description='fmin (Hz):', style={'description_width': 'initial'}, layout=widgets.Layout(width='400px'), continuous_update=False )
    fmax_slider = widgets.IntSlider( value=int(FMAX_DEFAULT), min=500, max=4200, step=100, description='fmax (Hz):', style={'description_width': 'initial'}, layout=widgets.Layout(width='400px'), continuous_update=False )
    info_label = widgets.Label(value="")
    warning_html = widgets.HTML( value=f"<b style='color:orange;'>Внимание:</b> Частота Найквиста для SR={SR_VIS}Hz равна <b style='color:cyan;'>{NYQUIST_FREQ:.0f} Гц</b>. Установка <b>fmax</b> выше этого значения использует диапазон частот, где нет информации об исходном сигнале." )
    params_box = widgets.VBox([ n_fft_dd, hop_length_slider, n_mels_slider, fmin_slider, fmax_slider, warning_html, info_label ])
    plot_output = widgets.Output()

    # --- Функции для связи виджетов (остаются те же, логика корректна) ---
    def _link_fft_hop(change):
        n_fft_val = n_fft_dd.value
        # hop_length не должен превышать n_fft
        hop_length_slider.max = n_fft_val
        hop_length_slider.value = min(hop_length_slider.value, n_fft_val) # Гарантируем, что hop <= n_fft
        win_ms = 1000 * n_fft_val / SR_VIS
        hop_ms = 1000 * hop_length_slider.value / SR_VIS
        info_label.value = f"(Window: ~{win_ms:.1f}ms, Hop: ~{hop_ms:.1f}ms)"
    n_fft_dd.observe(_link_fft_hop, names='value'); hop_length_slider.observe(_link_fft_hop, names='value')
    def _link_freqs(change):
        fmin_val = fmin_slider.value; fmax_slider.min = fmin_val + 100; fmax_slider.value = max(fmax_slider.value, fmax_slider.min)
    fmin_slider.observe(_link_freqs, names='value')

    # --- Обработчик изменений (остается тот же) ---
    def handle_melspec_vis_change(change):
        file_id = file_dd.value; n_fft_val = n_fft_dd.value; hop_length_val = hop_length_slider.value
        n_mels_val = n_mels_slider.value; fmin_val = fmin_slider.value; fmax_val = fmax_slider.value
        if not file_id:
            with plot_output: clear_output(wait=True); print("Выберите файл.")
            return
        audio_path = EXTRACTED_AUDIO_DIR / file_id
        if not audio_path.is_file():
            with plot_output: clear_output(wait=True); print(f"Ошибка: Файл не найден {audio_path}")
            return
        try:
            waveform, _ = librosa.load(audio_path, sr=SR_VIS, mono=True)
            mel_spec_db_res = calculate_interactive_melspec( waveform, SR_VIS, n_fft_val, hop_length_val, n_mels_val, fmin_val, fmax_val, POWER_DEFAULT, file_id_vis=file_id )
            with plot_output:
                clear_output(wait=True); print(f"Отображение: {file_id} (SR={SR_VIS}Hz)")
                plot_interactive_melspec( waveform, mel_spec_db_res, SR_VIS, hop_length_val, n_mels_val, fmin_val, fmax_val, n_fft_val, file_id, NYQUIST_FREQ )
        except Exception as e:
             with plot_output: clear_output(wait=True); print(f"Ошибка обработки {file_id}:\n{e}"); traceback.print_exc(limit=2)

    # --- Привязка обработчиков ко всем виджетам (остается та же) ---
    file_dd.observe(handle_melspec_vis_change, names='value'); n_fft_dd.observe(handle_melspec_vis_change, names='value'); hop_length_slider.observe(handle_melspec_vis_change, names='value'); n_mels_slider.observe(handle_melspec_vis_change, names='value'); fmin_slider.observe(handle_melspec_vis_change, names='value'); fmax_slider.observe(handle_melspec_vis_change, names='value')

    # --- Отображение UI и первый запуск (остается тем же) ---
    ui_melspec_tuning = widgets.VBox([ widgets.HTML(f"<b>Интерактивная настройка Мел-Спектрограммы (SR={SR_VIS}Hz, Маленькие n_fft)</b>"), file_dd, params_box, plot_output ])
    display(ui_melspec_tuning)
    _link_fft_hop(None); _link_freqs(None); handle_melspec_vis_change(None) # Первый запуск

    print(f"\n--- Ячейка 11: Интерактивная настройка Мел-Спектрограмм готова (Маленькие n_fft) ---")
print("-" * 50)
# =============================================================================

--- Ячейка 11: Интерактивная Настройка Мел-Спектрограмм (SR=8000 Гц, Маленькие n_fft) ---
>>> Исследуем n_fft = [32, 64, 128, 256] <<<
Параметры визуализации (Мел-Спектрограммы): SR=8000Hz (Найквист = 4000 Гц)
  Defaults: n_fft=256, hop=64, n_mels=16, fmin=0, fmax=4200



--- Ячейка 11: Интерактивная настройка Мел-Спектрограмм готова (Маленькие n_fft) ---
--------------------------------------------------


### Ячейка 12: Функция Полного Цикла Обучения

In [348]:
# =============================================================================
# Ячейка 12: Функция для Полного Цикла Обучения и Инференса (Исправлена заглушка MLflow)
# =============================================================================
print("--- Ячейка 12: Определение функции run_training_pipeline ---")

import torch, torch.nn as nn, torch.optim as optim
from torch.utils.data import DataLoader, random_split, Subset
from torch.optim.lr_scheduler import OneCycleLR
import numpy as np, pandas as pd, time, json, random, traceback
from pathlib import Path
from tqdm.notebook import tqdm
from typing import Union, Tuple, List, Dict, Optional

# --- Проверка и импорт MLflow или создание ЗАГЛУШКИ (ИСПРАВЛЕНО) ---
try:
    import mlflow
    import mlflow.pytorch
    MLFLOW_AVAILABLE = True
except ImportError:
    MLFLOW_AVAILABLE = False
    print("Предупреждение: mlflow не найден. Логирование экспериментов будет отключено.")
    class DummyRun:
        def __enter__(self): return self
        def __exit__(self, *args): pass
        class DummyRunInfo: info = type('obj', (object,), {'run_id': 'mlflow_disabled'})()
        info = DummyRunInfo()
    class DummyMLflow:
        def set_experiment(self, *args, **kwargs): pass
        def start_run(self, *args, **kwargs): return DummyRun()
        def log_param(self, *args, **kwargs): pass
        def log_params(self, *args, **kwargs): pass
        # === ИСПРАВЛЕНИЕ ЗДЕСЬ: Каждая функция на своей строке ===
        def log_metric(self, *args, **kwargs): pass
        def log_metrics(self, *args, **kwargs): pass
        # =====================================================
        def log_artifact(self, *args, **kwargs): pass
        def log_dict(self, *args, **kwargs): pass
        def set_tag(self, *args, **kwargs): pass
        def active_run(self): return None
        def end_run(self, status='FINISHED'): pass
    mlflow = DummyMLflow()


# --- Проверка глобальных зависимостей ---
def check_global_dependencies():
    required = ['MorseDataset', 'MorseRecognizer', 'collate_fn', 'train_epoch', 'validate_epoch',
                'ctc_greedy_decode', 'calculate_levenshtein', 'set_seed']
    missing = [r for r in required if not callable(globals().get(r))]
    if missing: raise NameError(f"Отсутствуют зависимости: {missing}. Убедитесь, что все ячейки выше выполнены.")

# --- Функция пайплайна ---
def run_training_pipeline(
    # Конфиги
    audio_config: Dict, model_config: Dict, train_config: Dict,
    # Данные/пути
    full_train_df: pd.DataFrame, test_df: pd.DataFrame, audio_dir: Path, base_output_dir: Path,
    # Словари/константы
    char_to_index: Dict[str, int], index_to_char: Dict[int, str],
    blank_idx: int, pad_idx: int, num_classes_ctc: int,
    # Управление
    device: torch.device, run_suffix: str, base_filename_suffix: str, # Используем полный базовый суффикс
    seed: int = 42, mlflow_experiment_name: str = "Morse_Experiment"
) -> Dict:
    """ Выполняет полный цикл: данные, обучение, сохранение, инференс. """
    check_global_dependencies()
    run_results = { "run_suffix": run_suffix, "status": "pending", "best_val_lev": float('inf'), "best_epoch": None,
                    "final_train_loss": float('inf'), "final_val_loss": float('inf'), "train_time_min": 0.0,
                    "infer_time_sec": 0.0, "model_path": None, "params_path": None, "submission_path": None, "error": None }
    print(f"\n{'='*20} Запуск: {run_suffix} {'='*20}")
    print(f"Audio Config: {json.dumps(audio_config)}")
    print(f"Model Config: {json.dumps(model_config)}") # Выводим конфиг модели
    print(f"Train Config: {json.dumps(train_config)}")

    # Формирование путей (используем полный базовый суффикс + суффикс запуска)
    output_filename_base = f"{base_filename_suffix}{run_suffix}"
    current_model_path = base_output_dir / f"model_{output_filename_base}.pth"
    current_params_path = base_output_dir / f"params_{output_filename_base}.json"
    current_submission_path = base_output_dir / f"submission_greedy_{output_filename_base}.csv"
    run_results.update({ "model_path": str(current_model_path), "params_path": str(current_params_path), "submission_path": str(current_submission_path) })
    print(f"  Пути: Model={current_model_path.name}, Params={current_params_path.name}, Sub={current_submission_path.name}")

    # Объявление переменных для finally
    model = None; criterion = None; optimizer = None; scheduler = None
    full_dataset = None; train_subset = None; val_subset = None
    train_loader = None; val_loader = None
    inference_model_instance = None; infer_test_dataset = None; test_loader_infer = None
    logits_infer = None; features_infer = None

    try:
        set_seed(seed)
        # --- 1. Подготовка Данных ---
        print("\n1. Подготовка данных...")
        if 'MorseDataset' not in globals(): raise NameError("Класс MorseDataset не определен!")
        full_dataset = MorseDataset( dataframe=full_train_df, audio_dir=audio_dir, char_to_index=char_to_index,
                                     audio_config=audio_config, model_input_feature_dim=model_config['input_feature_dim'],
                                     is_train=True )
        dataset_size = len(full_dataset);
        if dataset_size == 0: raise ValueError("Обучающий датасет пуст!")
        val_split_ratio = train_config['validation_split_ratio']
        val_size = int(np.floor(val_split_ratio * dataset_size)); train_size = dataset_size - val_size
        if train_size <= 0 or val_size <= 0: raise ValueError(f"Некорректное разделение: Train={train_size}, Val={val_size}")
        generator = torch.Generator().manual_seed(seed)
        train_subset, val_subset = random_split(full_dataset, [train_size, val_size], generator=generator)
        bs = train_config['batch_size']; nw = train_config['num_workers']; pm = (device.type == 'cuda')
        if 'collate_fn' not in globals(): raise NameError("Функция collate_fn не определена!")
        train_loader = DataLoader(train_subset, batch_size=bs, shuffle=True, collate_fn=collate_fn, num_workers=nw, pin_memory=pm)
        val_loader = DataLoader(val_subset, batch_size=bs*2, shuffle=False, collate_fn=collate_fn, num_workers=nw, pin_memory=pm)
        print(f"  Данные готовы: Train={len(train_subset)} ({len(train_loader)} батчей), Val={len(val_subset)} ({len(val_loader)} батчей).")

        # --- 2. Инициализация Модели, Loss, Optimizer, Scheduler ---
        print("\n2. Инициализация компонентов...")
        if 'MorseRecognizer' not in globals(): raise NameError("Класс MorseRecognizer не определен!")
        model = MorseRecognizer(num_classes_ctc=num_classes_ctc, **model_config).to(device) # Используем переданный model_config
        criterion = nn.CTCLoss(blank=blank_idx, reduction='mean', zero_infinity=True).to(device)
        optimizer_name = train_config.get('optimizer', 'AdamW').lower(); lr = train_config['learning_rate']; wd = train_config['weight_decay']
        if optimizer_name == 'adamw': optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=wd)
        elif optimizer_name == 'adam': optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=wd)
        else: print(f"Warning: Неизвестный optimizer '{optimizer_name}'. Используется AdamW."); optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=wd)
        print(f"  Model, Loss, Optimizer ({type(optimizer).__name__}) готовы.")
        # Scheduler
        total_batches_in_loader = len(train_loader); batches_per_epoch_run = train_config.get("batches_per_epoch", 0)
        steps_per_epoch = total_batches_in_loader
        if batches_per_epoch_run > 0 and batches_per_epoch_run < total_batches_in_loader: steps_per_epoch = batches_per_epoch_run
        if steps_per_epoch <= 0: raise ValueError(f"Steps_per_epoch ({steps_per_epoch}) должен быть > 0!")
        total_steps = train_config['num_epochs'] * steps_per_epoch
        div_f = train_config.get('div_factor', 25.0); final_div_f = train_config.get('final_div_factor', 1e4)
        scheduler = OneCycleLR(optimizer, max_lr=lr, total_steps=total_steps, pct_start=0.3, anneal_strategy='cos', div_factor=div_f, final_div_factor=final_div_f)
        print(f"  Scheduler OneCycleLR создан (total_steps={total_steps}, div_factor={div_f}, final_div_factor={final_div_f}).")

        # --- 3. Цикл Обучения с MLflow ---
        print("\n3. Запуск цикла обучения...")
        mlflow.set_experiment(mlflow_experiment_name)
        with mlflow.start_run(run_name=f"run_{run_suffix}") as current_run:
            mlflow_run_id = current_run.info.run_id; print(f"  MLflow Run ID: {mlflow_run_id}")
            run_results["mlflow_run_id"] = mlflow_run_id; run_results["status"] = "training"
            mlflow.log_param("run_suffix", run_suffix); mlflow.log_param("base_filename", base_filename_suffix)
            mlflow.log_params({"seed": seed, **train_config}); mlflow.log_dict(audio_config, "audio_config.json"); mlflow.log_dict(model_config, "model_config.json")
            mlflow.set_tag("status", "training")

            start_time_train = time.time(); best_val_lev = float('inf'); epochs_without_improvement = 0; best_epoch = None
            best_model_local_path_temp = base_output_dir / f"temp_best_model_{mlflow_run_id}.pth"

            for epoch in range(1, train_config['num_epochs'] + 1):
                print(f"\n--- Эпоха {epoch}/{train_config['num_epochs']} ---")
                if 'train_epoch' not in globals(): raise NameError("Функция train_epoch не определена!")
                avg_train_loss, avg_train_lev, avg_epoch_lr = train_epoch( model, train_loader, criterion, optimizer, scheduler, device, epoch, train_config['num_epochs'], index_to_char, blank_idx, pad_idx, train_config['gradient_clip_norm'], batches_per_epoch=batches_per_epoch_run )
                if 'validate_epoch' not in globals(): raise NameError("Функция validate_epoch не определена!")
                avg_val_loss, avg_val_lev, _ = validate_epoch( model, val_loader, criterion, device, index_to_char, blank_idx, pad_idx )
                run_results.update({ "final_train_loss": avg_train_loss, "final_val_loss": avg_val_loss })
                mlflow.log_metrics({ "train_loss": avg_train_loss, "train_levenshtein": avg_train_lev, "val_loss": avg_val_loss, "val_levenshtein": avg_val_lev, "learning_rate": avg_epoch_lr }, step=epoch)
                print(f"  Итоги Эпохи {epoch}: Train Loss={avg_train_loss:.4f}, Train Lev={avg_train_lev:.4f}, Val Loss={avg_val_loss:.4f}, Val Lev={avg_val_lev:.4f}")

                if np.isfinite(avg_val_lev) and avg_val_lev < best_val_lev:
                    print(f"  ✨ Val Lev улучшился: {best_val_lev:.4f} -> {avg_val_lev:.4f}. Сохранение..."); best_val_lev = avg_val_lev; best_epoch = epoch
                    run_results.update({ "best_val_lev": best_val_lev, "best_epoch": best_epoch })
                    try: torch.save(model.state_dict(), best_model_local_path_temp)
                    except Exception as save_err: print(f"  !!! Ошибка сохранения временной модели: {save_err} !!!")
                    epochs_without_improvement = 0
                else:
                    epochs_without_improvement += 1; print(f"  Val Lev не улучшился ({avg_val_lev:.4f} vs best {best_val_lev:.4f}). Без улучшений: {epochs_without_improvement}/{train_config['early_stopping_patience']}")
                if epochs_without_improvement >= train_config['early_stopping_patience']:
                    print(f"  ❗️ Ранняя остановка на эпохе {epoch}!"); mlflow.set_tag("status", "completed_early_stopping"); run_results["status"] = "completed_early_stopping"; break
            # --- Конец цикла по эпохам ---
            if run_results["status"] == "training": run_results["status"] = "completed"; mlflow.set_tag("status", "completed")
            train_duration_min = (time.time() - start_time_train) / 60; run_results["train_time_min"] = train_duration_min
            print(f"\n  Обучение завершено ({run_results['status']}) за {train_duration_min:.2f} мин. Лучший Val Lev: {best_val_lev:.4f} (Эпоха {best_epoch})")

            # --- 4. Сохранение Лучшей Модели и Параметров ---
            print("\n4. Сохранение артефактов...")
            if np.isfinite(best_val_lev): mlflow.log_metric("best_val_levenshtein", best_val_lev)
            if best_epoch is not None: mlflow.log_metric("best_epoch", best_epoch)
            mlflow.log_metric("training_time_min", train_duration_min)

            model_saved_final = False
            if best_model_local_path_temp.exists() and np.isfinite(best_val_lev):
                try:
                    current_model_path.parent.mkdir(parents=True, exist_ok=True); current_model_path.unlink(missing_ok=True)
                    best_model_local_path_temp.rename(current_model_path); print(f"  Лучшая модель сохранена: {current_model_path.name}")
                    mlflow.log_artifact(str(current_model_path), artifact_path="model"); model_saved_final = True
                except Exception as e: print(f"  !!! Ошибка перемещения/логирования модели: {e} !!!")
            else: print("  !!! Лучшая модель не сохранена."); mlflow.set_tag("model_saved", "False")

            final_params = { 'audio_config': audio_config, 'model_config': model_config, 'train_config': train_config,
                             'char_map': { 'char_to_index': char_to_index, 'index_to_char': {str(k): v for k, v in index_to_char.items()},
                                           'BLANK_IDX': blank_idx, 'PAD_IDX': pad_idx, 'NUM_CLASSES_CTC': num_classes_ctc },
                             'results': { k: v for k, v in run_results.items() if k not in ['model_path', 'params_path', 'submission_path', 'mlflow_run_id'] and (np.isfinite(v) if isinstance(v, float) else True) } }
            try:
                current_params_path.parent.mkdir(parents=True, exist_ok=True); current_params_path.unlink(missing_ok=True)
                with open(current_params_path, 'w', encoding='utf-8') as f: json.dump(final_params, f, indent=4, ensure_ascii=False)
                print(f"  Параметры сохранены: {current_params_path.name}"); mlflow.log_artifact(str(current_params_path), artifact_path="config")
            except Exception as e: print(f"  !!! Ошибка сохранения/логирования параметров: {e} !!!")
        # --- Конец MLflow run ---

        # --- 5. Инференс на Тестовых Данных ---
        print("\n5. Запуск инференса...")
        infer_duration_sec = 0.0
        if model_saved_final:
            infer_start_time = time.time()
            try:
                 print(f"  Загрузка: {current_model_path.name}, {current_params_path.name}")
                 with open(current_params_path, 'r', encoding='utf-8') as f: loaded_params_inf = json.load(f)
                 loaded_audio_config_inf = loaded_params_inf['audio_config']; loaded_model_config_inf = loaded_params_inf['model_config']
                 loaded_char_map_inf = loaded_params_inf['char_map']; loaded_index_to_char_inf = {int(k): v for k, v in loaded_char_map_inf['index_to_char'].items()}
                 loaded_blank_idx_inf = loaded_char_map_inf['BLANK_IDX']; loaded_num_classes_inf = loaded_char_map_inf['NUM_CLASSES_CTC']

                 if 'MorseRecognizer' not in globals(): raise NameError("Класс MorseRecognizer не определен для инференса!")
                 inference_model_instance = MorseRecognizer(num_classes_ctc=loaded_num_classes_inf, **loaded_model_config_inf).to(device)
                 try: inference_model_instance.load_state_dict(torch.load(current_model_path, map_location=device, weights_only=True))
                 except TypeError: print("   Warning: weights_only=True не поддерживается."); inference_model_instance.load_state_dict(torch.load(current_model_path, map_location=device))
                 inference_model_instance.eval(); print(f"  Модель для инференса загружена.")

                 print(f"  Создание тестового DataLoader...");
                 if 'MorseDataset' not in globals(): raise NameError("Класс MorseDataset не определен для инференса!")
                 infer_test_dataset = MorseDataset( dataframe=test_df, audio_dir=audio_dir, char_to_index=loaded_char_map_inf['char_to_index'],
                                                    audio_config=loaded_audio_config_inf, model_input_feature_dim=loaded_model_config_inf['input_feature_dim'], is_train=False )
                 infer_bs = train_config.get('batch_size', 16) * 4
                 if 'collate_fn' not in globals(): raise NameError("Функция collate_fn не определена для инференса!")
                 test_loader_infer = DataLoader(infer_test_dataset, batch_size=infer_bs, shuffle=False, collate_fn=collate_fn, num_workers=0, pin_memory=(device.type == 'cuda') )

                 print(f"  Предсказание..."); predictions: Dict[str, str] = {}
                 if 'ctc_greedy_decode' not in globals(): raise NameError("Функция ctc_greedy_decode не определена для инференса!")
                 with torch.no_grad():
                     pbar_infer = tqdm(test_loader_infer, desc="Инференс", leave=False, ncols=1000)
                     for batch_data_infer in pbar_infer:
                         if batch_data_infer is None: continue
                         features_infer, file_ids_infer, _, _ = batch_data_infer
                         if features_infer is None or file_ids_infer is None or len(features_infer) == 0: continue
                         features_infer = features_infer.to(device, non_blocking=True)
                         try:
                             logits_infer = inference_model_instance(features_infer)
                             decoded_batch_infer = ctc_greedy_decode(logits_infer, loaded_index_to_char_inf, loaded_blank_idx_inf)
                             for file_id, pred_text in zip(file_ids_infer, decoded_batch_infer): predictions[file_id] = pred_text
                         except Exception as e_inf_batch: print(f"\nОшибка инференса батча: {e_inf_batch}"); [predictions.update({fid:"ERROR_INFER"}) for fid in file_ids_infer]

                 infer_duration_sec = time.time() - infer_start_time; run_results["infer_time_sec"] = infer_duration_sec
                 print(f"  Инференс завершен за {infer_duration_sec:.2f} сек. Предсказаний: {len(predictions)}/{len(test_df)}")

                 if predictions:
                     print(f"  Формирование submission: {current_submission_path.name}")
                     submission_df = pd.DataFrame({'id': test_df['id']}); submission_df['message'] = submission_df['id'].map(predictions).fillna("ERROR_MISSING")
                     current_submission_path.unlink(missing_ok=True)
                     submission_df.to_csv(current_submission_path, index=False); print(f"  Submission сохранен."); mlflow.log_artifact(str(current_submission_path), artifact_path="submission")
                 else: print("  Предсказания не сгенерированы."); mlflow.set_tag("submission_generated", "False")
            except Exception as e_infer: print(f"  !!! Ошибка инференса: {e_infer} !!!"); traceback.print_exc(limit=2); run_results["error"] = f"Inference Error: {e_infer}"; run_results["status"] = "failed_inference"; mlflow.set_tag("status", "failed_inference")
            finally:
                 try: del inference_model_instance
                 except NameError: pass
                 try: del test_loader_infer
                 except NameError: pass
                 try: del infer_test_dataset
                 except NameError: pass
                 if 'logits_infer' in locals(): del logits_infer
                 if 'features_infer' in locals(): del features_infer
                 if torch.cuda.is_available(): torch.cuda.empty_cache()
        else: print("  Инференс пропущен (лучшая модель не сохранена)."); run_results["status"] += "_no_inference"; mlflow.set_tag("inference_skipped", "True")

    except Exception as e_main: print(f"!!! КРИТИЧЕСКАЯ ОШИБКА в {run_suffix}: {e_main} !!!"); traceback.print_exc(); run_results["error"] = str(e_main); run_results["status"] = "failed_critical";
    try: mlflow.set_tag("status", "failed_critical"); mlflow.set_tag("error", str(e_main)[:250])
    except: pass

    finally:
        try: del model
        except NameError: pass
        try: del criterion
        except NameError: pass
        try: del optimizer
        except NameError: pass
        try: del scheduler
        except NameError: pass
        if 'full_dataset' in locals(): del full_dataset
        if 'train_subset' in locals(): del train_subset
        if 'val_subset' in locals(): del val_subset
        if 'train_loader' in locals(): del train_loader
        if 'val_loader' in locals(): del val_loader
        if torch.cuda.is_available(): torch.cuda.empty_cache()
        print(f"{'='*20} Завершение: {run_suffix} {'='*20}\n")
    return run_results

print("--- Функция run_training_pipeline (Исправлена заглушка MLflow) определена ---")
print("-" * 50)
# =============================================================================

--- Ячейка 12: Определение функции run_training_pipeline ---
--- Функция run_training_pipeline (Исправлена заглушка MLflow) определена ---
--------------------------------------------------


### Ячейка 13: Основной Цикл Обучения

In [ ]:
# -*- coding: utf-8 -*-
# =============================================================================
# Ячейка 13: Цикл Запуска Обучения (Тест n_mels = 8, 12 с hop_length=64, УПРОЩЕННАЯ модель) - ОБНОВЛЕНА
# =============================================================================
print(f"--- Ячейка 13: Запуск Цикла Обучения (Тест n_mels = 8, 12, hop={HOP_LENGTH_CONST}, УПРОЩЕННАЯ модель) ---")

# Импорты, необходимые для этой ячейки
import time, pandas as pd, numpy as np, json, random, copy, traceback
from pathlib import Path; import torch
from IPython.display import display

# --- Проверка глобальных зависимостей ---
required_globals_loop = [
    'train_df', 'test_df', 'EXTRACTED_AUDIO_DIR',
    'char_to_index', 'index_to_char', 'BLANK_IDX', 'PAD_IDX', 'NUM_CLASSES_CTC',
    'device', 'OUTPUT_DIR',
    'BASE_FILENAME_SUFFIX_TEMPLATE_SIMPLE', # <-- Используем шаблон для УПРОЩЕННОЙ модели
    'SEED', 'run_training_pipeline',
    'AUDIO_CONFIG_MELSPEC',         # Базовый конфиг аудио
    'MODEL_CONFIG_SIMPLE',          # <-- Используем конфиг УПРОЩЕННОЙ модели
    'TRAIN_CONFIG_MELSPEC',         # Конфиг обучения (с уменьшенными эпохами)
    'HOP_LENGTH_CONST',             # Константа hop_length
    'mlflow'
]
missing_globals_loop = [v for v in required_globals_loop if v not in globals() or globals().get(v) is None]
if missing_globals_loop: raise NameError(f"Отсутствуют переменные/функции или они None: {missing_globals_loop}")

# ============================================================================
# === БЛОК 1: Определение Параметров для Цикла Экспериментов ===
# ============================================================================
print("\n1. Определение параметров для цикла экспериментов...")

N_MELS_TO_TEST = [12] # <-- Пробуем [8, 12] и добавим [16, 24, 32] для сравнения
print(f"  Значения n_mels для теста: {N_MELS_TO_TEST}")
print(f"  Константа hop_length: {HOP_LENGTH_CONST}")
print(f"  Используется УПРОЩЕННАЯ модель для калибровки.")

# --- 2. Копирование Базовой Конфигурации Обучения ---
print("\n2. Копирование конфигурации обучения (с уменьшенными эпохами)...")
fixed_train_config = TRAIN_CONFIG_MELSPEC.copy() # Используем конфиг из Ячейки 2 (уже с уменьшенными эпохами)

# --- 3. Инициализация Списка для Результатов ---
all_run_results_list: List[Dict] = []

# ============================================================================
# === БЛОК 2: Цикл запуска обучения по значениям n_mels ===
# ============================================================================
print(f"\n--- Начало цикла запусков ({len(N_MELS_TO_TEST)} итераций) ---")
overall_start_time = time.time()
# === Имя эксперимента MLflow ===
MLFLOW_EXPERIMENT_NAME = f"Morse_MelSpec_NMels_Calibration_SimpleModel_Hop{HOP_LENGTH_CONST}" # <-- ОБНОВЛЕНО
# ==============================
print(f"MLflow эксперимент: '{MLFLOW_EXPERIMENT_NAME}'")

# --- Надежное завершение ЛЮБОГО активного MLflow run перед циклом ---
print("\n--- Попытка завершить ЛЮБОЙ активный MLflow run перед циклом ---")
try:
    if mlflow.active_run(): active_run_id = mlflow.active_run().info.run_id; print(f"!!! Обнаружен активный MLflow run ({active_run_id}). Принудительно завершаем... !!!"); mlflow.end_run(); time.sleep(1); print("--- Проверка: Активный MLflow run отсутствует." if not mlflow.active_run() else "!!! ПРЕДУПРЕЖДЕНИЕ: MLflow run ВСЕ ЕЩЕ АКТИВЕН! !!!")
    else: print("--- Активный MLflow run не обнаружен. ---")
except Exception as e_check_run: print(f"Предупреждение: Ошибка при проверке/завершении активного MLflow run: {e_check_run}")
# -------------------------------------------------------------

# --- Основной цикл по значениям n_mels ---
for i, n_mels_current in enumerate(N_MELS_TO_TEST):

    print(f"\n{'='*10} Запуск {i+1}/{len(N_MELS_TO_TEST)}: n_mels = {n_mels_current}, hop_length = {HOP_LENGTH_CONST}, Simple Model {'='*10}")

    # --- Создание специфичных конфигураций для текущего запуска ---
    current_audio_config = AUDIO_CONFIG_MELSPEC.copy()
    current_audio_config['n_mels'] = n_mels_current
    current_audio_config['hop_length'] = HOP_LENGTH_CONST # Убеждаемся еще раз

    current_model_config = MODEL_CONFIG_SIMPLE.copy() # <-- Используем УПРОЩЕННУЮ модель
    current_model_config['input_feature_dim'] = n_mels_current # Ключевое изменение для модели

    print(f"  Аудио параметры: {json.dumps(current_audio_config)}")
    print(f"  Параметры модели (Упрощенной): {json.dumps(current_model_config)}")
    print(f"{'-'*30}")

    # --- Генерация уникального базового суффикса для этого запуска ---
    current_base_filename_suffix = BASE_FILENAME_SUFFIX_TEMPLATE_SIMPLE.format(n_mels=n_mels_current) # <-- Используем шаблон SIMPLE
    current_run_suffix = f"_nMels{n_mels_current}_Simple" # Добавляем "_Simple" для ясности

    # === Дополнительная попытка завершить run ПЕРЕД start_run ВНУТРИ цикла ===
    print(f"--- Проверка/Завершение MLflow run перед итерацией {i+1}... ---")
    try:
        if mlflow.active_run(): lingering_run_id = mlflow.active_run().info.run_id; print(f"!!! ПРЕДУПРЕЖДЕНИЕ: Обнаружен активный run ({lingering_run_id}) ПЕРЕД start_run в итерации {i+1}. Попытка завершения... !!!"); mlflow.end_run(); time.sleep(0.5); print(f"--- Зависший run {lingering_run_id} успешно завершен." if not mlflow.active_run() else f"!!! ОШИБКА: Run {lingering_run_id} все еще активен! !!!")
        else: print(f"--- Активный run не найден перед итерацией {i+1}. Продолжаем. ---")
    except Exception as e_inner_end: print(f"Предупреждение: Ошибка при попытке end_run внутри цикла: {e_inner_end}")
    # ======================================================================

    # --- Вызов основной функции обучения/инференса ---
    try:
        print(f"--- Вызов run_training_pipeline для n_mels={n_mels_current} (Simple Model) ---")
        run_result_dict = run_training_pipeline(
            audio_config=current_audio_config,
            model_config=current_model_config, # <-- Передаем УПРОЩЕННУЮ модель
            train_config=fixed_train_config, # Используем общий конфиг обучения (с уменьшенными эпохами)
            full_train_df=train_df, test_df=test_df, audio_dir=EXTRACTED_AUDIO_DIR, base_output_dir=OUTPUT_DIR,
            char_to_index=char_to_index, index_to_char=index_to_char, blank_idx=BLANK_IDX, pad_idx=PAD_IDX, num_classes_ctc=NUM_CLASSES_CTC,
            device=device,
            run_suffix=current_run_suffix, # Суффикс для MLflow
            base_filename_suffix=current_base_filename_suffix, # Полный суффикс для файлов
            seed=SEED,
            mlflow_experiment_name=MLFLOW_EXPERIMENT_NAME
        )
        print(f"--- Успешное завершение run_training_pipeline для n_mels={n_mels_current} (Simple Model) ---")

    except Exception as e_pipeline:
        print(f"!!! КРИТИЧЕСКАЯ ОШИБКА ПРИ ВЫЗОВЕ ПАЙПЛАЙНА для n_mels={n_mels_current} (Simple Model) !!!")
        print(f"Ошибка: {e_pipeline}"); traceback.print_exc()
        run_result_dict = { "run_suffix": current_run_suffix, "status": "failed_launch", "best_val_lev": float('inf'),
                            "error": f"Pipeline launch error: {e_pipeline}", "n_mels": n_mels_current }

    # Добавляем параметры этого запуска в результаты
    run_result_dict['n_mels'] = n_mels_current
    run_result_dict['hop_length'] = HOP_LENGTH_CONST
    run_result_dict['config_name'] = f"nMels{n_mels_current}_Hop{HOP_LENGTH_CONST}_Simple" # Имя конфигурации
    run_result_dict['trimming_status'] = f"ON (db={current_audio_config.get('trim_top_db', 'N/A')})" if current_audio_config.get('apply_trimming', False) else "OFF"
    all_run_results_list.append(run_result_dict)

    print(f"--- Завершение обработки Запуска {i+1}/{len(N_MELS_TO_TEST)} (n_mels={n_mels_current}, Simple Model) ---")

# --- Конец цикла запусков ---
# ============================================================================
# === КОНЕЦ БЛОКА 2 ===
# ============================================================================

overall_end_time = time.time(); total_duration_hours = (overall_end_time - overall_start_time) / 3600
print(f"\n--- Цикл запусков (УПРОЩЕННАЯ модель) завершен за {total_duration_hours:.2f} часов ---")

# ============================================================================
# === БЛОК 3: Анализ и вывод итогов ===
# ============================================================================
print(f"\n--- Итоги Теста n_mels = {N_MELS_TO_TEST} (hop_length={HOP_LENGTH_CONST}, УПРОЩЕННАЯ модель) ---") # <-- ОБНОВЛЕНО
if not all_run_results_list: print("Нет результатов для анализа.")
else:
    results_df = pd.DataFrame(all_run_results_list)
    # Определяем колонки для сохранения и отображения
    required_cols = ['config_name', 'n_mels', 'hop_length', 'best_val_lev', 'best_epoch', 'final_train_loss', 'final_val_loss',
                     'status', 'train_time_min', 'infer_time_sec', 'error', 'run_suffix', 'mlflow_run_id']
    available_cols = [col for col in required_cols if col in results_df.columns]
    results_df = results_df[available_cols]
    # Сортируем по лучшей метрике валидации
    if 'best_val_lev' in results_df.columns: results_df = results_df.sort_values(by='best_val_lev', ascending=True, na_position='last')
    else: print("Предупреждение: Колонка 'best_val_lev' отсутствует, сортировка невозможна.")

    # Настройки отображения Pandas
    pd.set_option('display.max_rows', 100); pd.set_option('display.max_columns', 20);
    pd.set_option('display.width', 180); pd.set_option('display.float_format', '{:.4f}'.format)

    print("\nСводная таблица результатов (УПРОЩЕННАЯ модель):")
    display_cols = ['config_name', 'n_mels', 'best_val_lev', 'best_epoch', 'status', 'train_time_min', 'error']
    display_cols_present = [col for col in display_cols if col in results_df.columns]
    display(results_df[display_cols_present].head(len(results_df)))

    # Вывод лучшего результата
    if 'best_val_lev' in results_df.columns and not results_df.empty:
        best_run = results_df.iloc[0]
        if pd.notna(best_run['best_val_lev']) and np.isfinite(best_run['best_val_lev']):
            best_n_mels_simple = best_run.get('n_mels', 'N/A')
            print(f"\n--- Лучший результат калибровки (УПРОЩЕННАЯ модель) ---")
            print(f"  n_mels: {best_n_mels_simple}")
            print(f"  Best Val Levenshtein: {best_run['best_val_lev']:.4f} (Эпоха {best_run.get('best_epoch', 'N/A')})")
            print(f"\n>>> РЕКОМЕНДАЦИЯ: Провести финальное обучение ПОЛНОЙ модели с n_mels = {best_n_mels_simple} (и hop_length={HOP_LENGTH_CONST}). <<<")
        else: print("\nНе удалось найти валидный лучший результат в этом цикле калибровки.")
    else: print("\nНе удалось определить лучший результат калибровки.")

    # Сохранение полной таблицы результатов
    results_csv_path = OUTPUT_DIR / f"melspec_nmels_calibration_simplemodel_hop{HOP_LENGTH_CONST}_results.csv" # <-- ОБНОВЛЕНО
    try: results_df.to_csv(results_csv_path, index=False); print(f"\nПолные результаты калибровки сохранены в CSV: {results_csv_path}")
    except Exception as e: print(f"\nНе удалось сохранить результаты калибровки в CSV: {e}")
# ============================================================================
# === КОНЕЦ БЛОКА 3 ===
# ============================================================================

print(f"\n--- Ячейка 13: Завершена (Тест n_mels = {N_MELS_TO_TEST}, hop_length={HOP_LENGTH_CONST}, УПРОЩЕННАЯ модель) ---")
print("-" * 50)
# =============================================================================

--- Ячейка 13: Запуск Цикла Обучения (Тест n_mels = 8, 12, hop=64, УПРОЩЕННАЯ модель) ---

1. Определение параметров для цикла экспериментов...
  Значения n_mels для теста: [12]
  Константа hop_length: 64
  Используется УПРОЩЕННАЯ модель для калибровки.

2. Копирование конфигурации обучения (с уменьшенными эпохами)...

--- Начало цикла запусков (1 итераций) ---
MLflow эксперимент: 'Morse_MelSpec_NMels_Calibration_SimpleModel_Hop64'

--- Попытка завершить ЛЮБОЙ активный MLflow run перед циклом ---
!!! Обнаружен активный MLflow run (e99ccd86c412461dbc19c5fbc3a2cc88). Принудительно завершаем... !!!
Предупреждение: Ошибка при проверке/завершении активного MLflow run: The run e99ccd86c412461dbc19c5fbc3a2cc88 must be in 'active' lifecycle_stage.

========== Запуск 1/1: n_mels = 12, hop_length = 64, Simple Model ==========
  Аудио параметры: {"feature_type": "melspec", "sample_rate": 8000, "n_fft": 128, "hop_length": 64, "n_mels": 12, "fmin": 0.0, "fmax": 4200.0, "power": 2.0, "apply_trimming

Эпоха 1/10 [Тренировка]:   0%|                                                                                …

   [Валидация]:   0%|                                                                                         …

  Итоги Эпохи 1: Train Loss=4.2039, Train Lev=8.9545, Val Loss=4.0138, Val Lev=8.8780
  ✨ Val Lev улучшился: inf -> 8.8780. Сохранение...

--- Эпоха 2/10 ---


Эпоха 2/10 [Тренировка]:   0%|                                                                                …

...

  Итоги Эпохи 3: Train Loss=0.6334, Train Lev=1.3906, Val Loss=0.5054, Val Lev=1.1553
  ✨ Val Lev улучшился: 1.4747 -> 1.1553. Сохранение...

--- Эпоха 4/10 ---


Эпоха 4/10 [Тренировка]:   0%|                                                                                …

   [Валидация]:   0%|                                                                                         …

  Итоги Эпохи 4: Train Loss=0.5446, Train Lev=1.2291, Val Loss=0.4294, Val Lev=0.9890
  ✨ Val Lev улучшился: 1.1553 -> 0.9890. Сохранение...

--- Эпоха 5/10 ---


Эпоха 5/10 [Тренировка]:   0%|                                                                                …